In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Simulation minimale d'un système FitzHugh–Nagumo isolé (une seule cellule) sans diffusion.
L'objectif est de vérifier que l'injection d'un mini-pic de cAMP entraîne une réponse
excitable : A augmente puis retombe, en fonction des paramètres du modèle.

Auteur : Ton Nom
Date   : 2025-02-13
"""

import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Paramètres du modèle FHN
# -------------------------------
a = 0.1         # Intensité du terme de stimulation (dans I_S)
Kd = 5     # Constante de dissociation pour le cAMP
gamma = 0.1    # Facteur de couplage entre A et R
c0 = 0.5        # Terme constant influençant la récupération (R)
epsilon = 0.2   # Facteur d'échelle pour la mise à jour de R
sigma = 0.0     # Amplitude du bruit (optionnel, ici désactivé)

# -------------------------------
# Paramètres de la simulation
# -------------------------------
dt = 0.001       # Pas de temps (en unité de temps)
T_total = 100.0  # Temps total de simulation
n_steps = int(T_total / dt)

# Conditions initiales (tu peux les modifier)
A = 0.0
R = 0.0

# Fonction pour calculer le terme d'excitation I_S à partir du signal de cAMP
def I_S(signal):
    return a * np.log1p(signal / Kd)

# -------------------------------
# Définition du signal de cAMP
# -------------------------------
# On définit un vecteur "signal" qui sera nul partout sauf pendant une courte durée (mini-pic)
signal = np.zeros(n_steps)
t_inject = 1.0         # Temps d'injection (en unités de temps)
inject_duration = 0.1  # Durée de l'injection
inject_start = int(t_inject / dt)
inject_steps = int(inject_duration / dt)
signal[inject_start:inject_start + inject_steps] = 1.0  # Valeur du pic

# -------------------------------
# Stockage des valeurs pour tracer
# -------------------------------
A_vals = np.zeros(n_steps)
R_vals = np.zeros(n_steps)
time_arr = np.linspace(0, T_total, n_steps)

# -------------------------------
# Intégration numérique (méthode d'Euler)
# -------------------------------
for i in range(n_steps):
    A_vals[i] = A
    R_vals[i] = R
    # Calcul du terme d'excitation à partir du signal actuel
    current_I_S = I_S(signal[i])
    # Équations différentielles du modèle FHN
    dA = (A - (A ** 3) / 3 - R + current_I_S) * dt
    dR = epsilon * (A - gamma * R + c0) * dt
    # Ajout éventuellement du bruit (ici sigma = 0 donc pas de bruit)
    # dA += sigma * np.sqrt(dt) * np.random.randn()
    # Mise à jour des états
    A += dA
    R += dR

# -------------------------------
# Tracé des résultats
# -------------------------------
plt.figure(figsize=(8, 5))
plt.plot(time_arr, A_vals, label='A (activateur)', lw=2)
plt.plot(time_arr, R_vals, label='R (répresseur)', lw=2)
plt.xlabel('Temps')
plt.ylabel('Valeur')
plt.title("Simulation FHN isolé : réponse à un mini-pic de cAMP")
plt.legend()
plt.grid(True)
plt.show()

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/1658763348.py:86: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Paramètres du modèle FHN
# -------------------------------
a = 0.1         # Intensité du terme de stimulation dans I_S
Kd = 5          # Constante de dissociation pour le cAMP
gamma = 0.1     # Facteur de couplage entre A et R
c0 = 0.2        # Terme constant influençant la récupération (R)
epsilon = 0.08   # Facteur d'échelle pour la mise à jour de R
sigma = 0.1     # Amplitude du bruit (désactivé ici)

# -------------------------------
# Paramètres de production de cAMP
# -------------------------------
D = 2500.0      # Quantité de cAMP produite lorsque A dépasse le seuil
af = -1.2        # Seuil d'activation pour la production de cAMP

# -------------------------------
# Paramètres de la simulation
# -------------------------------
dt = 0.001       # Pas de temps
T_total = 100.0  # Temps total de simulation
n_steps = int(T_total / dt)

# Conditions initiales
A = 0.0
R = 0.0

# Fonction pour calculer le terme d'excitation I_S à partir du signal de cAMP externe
def I_S(signal):
    return a * np.log1p(signal / Kd)

# -------------------------------
# Définition du signal externe de cAMP
# -------------------------------
signal = np.zeros(n_steps)
t_inject = 1.0         # Temps d'injection
inject_duration = 0.1  # Durée de l'injection
inject_start = int(t_inject / dt)
inject_steps = int(inject_duration / dt)
signal[inject_start:inject_start + inject_steps] = 1.0  # Injection d'un mini-pic

# -------------------------------
# Stockage des valeurs pour tracer
# -------------------------------
A_vals = np.zeros(n_steps)
R_vals = np.zeros(n_steps)
production_vals = np.zeros(n_steps)  # Pour la production de cAMP
time_arr = np.linspace(0, T_total, n_steps)

# -------------------------------
# Intégration numérique (méthode d'Euler)
# -------------------------------
for i in range(n_steps):
    A_vals[i] = A
    R_vals[i] = R
    # Calcul du terme d'excitation basé sur le signal externe
    current_I_S = I_S(signal[i])
    
    # Équations du modèle FHN
    dA = (A - (A ** 3) / 3 - R + current_I_S) * dt
    dR = epsilon * (A - gamma * R + c0) * dt
    
    # Mise à jour des états
    A += dA
    R += dR
    
    # Calcul de la production de cAMP (on considère que la cellule produit D si A > af)
    production_vals[i] = D if A > af else 0

# -------------------------------
# Tracé avec deux axes y distincts
# -------------------------------
fig, ax1 = plt.subplots(figsize=(10, 6))

# Axe 1 pour A et R
ax1.plot(time_arr, A_vals, label='A (activateur)', color='blue', lw=2)
ax1.plot(time_arr, R_vals, label='R (répresseur)', color='green', lw=2)
ax1.set_xlabel('Temps')
ax1.set_ylabel('A et R (valeurs)')
ax1.grid(True)

# Axe 2 pour la production de cAMP (échelle différente)
ax2 = ax1.twinx()
ax2.plot(time_arr, production_vals, label='cAMP produit', color='red', lw=2, linestyle='--')
ax2.set_ylabel('cAMP produit (quantité)')

# Combinaison des légendes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.title("Simulation FHN isolé avec production de cAMP")
plt.show()

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/402240315.py:96: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Paramètres du modèle FHN
# -------------------------------
a = 0.1         # Intensité du terme de stimulation dans I_S
Kd = 5          # Constante de dissociation pour le cAMP
gamma = 0.15     # Facteur de couplage entre A et R
c0 = 0.1       # Terme constant influençant la récupération (R)
epsilon = 0.088   # Facteur d'échelle pour la mise à jour de R
sigma = 0.1     # Amplitude du bruit (désactivé ici)

# -------------------------------
# Paramètres de production de cAMP
# -------------------------------
D = 2500.0      # Quantité de cAMP produite lorsque A dépasse le seuil
af = -1.2        # Seuil d'activation pour la production de cAMP

# -------------------------------
# Paramètres de la simulation
# -------------------------------
dt = 0.001       # Pas de temps
T_total = 100.0  # Temps total de simulation
n_steps = int(T_total / dt)

# Conditions initiales
A = -2
R = 0.67

# Fonction pour calculer le terme d'excitation I_S à partir du signal de cAMP externe
def I_S(signal):
    return a * np.log1p(signal / Kd)

# -------------------------------
# Définition du signal externe de cAMP
# -------------------------------
signal = np.zeros(n_steps)
t_inject = 5.0         # Temps d'injection
inject_duration = 5  # Durée de l'injection
inject_start = int(t_inject / dt)
inject_steps = int(inject_duration / dt)
signal[inject_start:inject_start + inject_steps] = 10.0  # Injection d'un mini-pic

# -------------------------------
# Stockage des valeurs pour tracer
# -------------------------------
A_vals = np.zeros(n_steps)
R_vals = np.zeros(n_steps)
production_vals = np.zeros(n_steps)  # Pour la production de cAMP
time_arr = np.linspace(0, T_total, n_steps)

# -------------------------------
# Intégration numérique (méthode d'Euler)
# -------------------------------
for i in range(n_steps):
    A_vals[i] = A
    R_vals[i] = R
    # Calcul du terme d'excitation basé sur le signal externe
    current_I_S = I_S(signal[i])
    
    # Équations du modèle FHN
    dA = (A - (A ** 3) / 3 - R + current_I_S) * dt
    dR = epsilon * (A - gamma * R + c0) * dt
    
    # Mise à jour des états
    A += dA
    R += dR
    
    # Calcul de la production de cAMP (on considère que la cellule produit D si A > af)
    production_vals[i] = D if A > af else 0

# -------------------------------
# Tracé avec trois axes y distincts
# -------------------------------
fig, ax1 = plt.subplots(figsize=(10, 6))

# Axe 1 pour A et R
ax1.plot(time_arr, A_vals, label='A (activateur)', color='blue', lw=2)
ax1.plot(time_arr, R_vals, label='R (répresseur)', color='green', lw=2)
ax1.set_xlabel('Temps')
ax1.set_ylabel('A et R (valeurs)')
ax1.grid(True)

# Axe 2 pour la production de cAMP
ax2 = ax1.twinx()
ax2.plot(time_arr, production_vals, label='cAMP produit', color='red', lw=2, linestyle='--')
ax2.set_ylabel('cAMP produit (quantité)')

# Axe 3 pour le signal externe
ax3 = ax1.twinx()
# Décaler l'axe 3 vers la droite
ax3.spines["right"].set_position(("axes", 1.15))
ax3.plot(time_arr, signal, label='Signal externe', color='purple', lw=2, linestyle=':')
ax3.set_ylabel('Signal externe')
ax3.tick_params(axis='y', labelcolor='purple')

# Combiner les légendes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
ax1.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, loc='upper right')

plt.title("Simulation FHN isolé avec production de cAMP et signal externe")
plt.show()

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/1419008860.py:105: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# -------------------------------
# Paramètres communs du modèle FHN
# -------------------------------
a = 0.1         # Intensité du terme de stimulation dans I_S
Kd = 5          # Constante de dissociation pour le cAMP
gamma = 0.15    # Facteur de couplage entre A et R
c0 = 0.1        # Terme constant influençant la récupération (R)
sigma = 0.0     # Amplitude du bruit (désactivé ici)

# Paramètres de production de cAMP
D = 2500.0      # Quantité de cAMP produite lorsque A dépasse le seuil
af = 1.2        # Seuil d'activation pour la production de cAMP

# Paramètres du signal externe
t_inject = 1.0         # Temps d'injection
inject_duration = 5    # Durée de l'injection
signal_value = 10.0    # Amplitude du signal externe pendant l'injection

# Paramètres de la simulation
dt = 0.001       # Pas de temps
T_total = 100.0  # Temps total de simulation
n_steps = int(T_total / dt)
time_arr = np.linspace(0, T_total, n_steps)

# Fonction pour calculer le terme d'excitation I_S
def I_S(signal):
    return a * np.log1p(signal / Kd)

# Liste des valeurs de ε à tester (balayage paramétrique)
epsilon_values = np.linspace(0.05, 0.2, 5)
results = {}  # Pour stocker la période moyenne de chaque simulation

plt.figure(figsize=(10, 6))

# Balayage sur ε
for eps in epsilon_values:
    # Réinitialisation des variables d'état pour chaque simulation
    A = 0.0
    R = 0.0
    A_vals = np.zeros(n_steps)
    
    # Définition du signal externe : nul sauf pendant l'injection
    signal = np.zeros(n_steps)
    inject_start = int(t_inject / dt)
    inject_steps = int(inject_duration / dt)
    signal[inject_start:inject_start + inject_steps] = signal_value

    # Intégration numérique par la méthode d'Euler
    for i in range(n_steps):
        A_vals[i] = A
        current_I_S = I_S(signal[i])
        dA = (A - (A**3)/3 - R + current_I_S) * dt
        dR = eps * (A - gamma * R + c0) * dt
        A += dA
        R += dR

    # Détection des pics dans A après l'injection
    peaks, properties = find_peaks(A_vals, height=af)
    if len(peaks) > 1:
        periods = np.diff(time_arr[peaks])
        avg_period = np.mean(periods)
    else:
        avg_period = np.nan  # Si aucun pic ou un seul pic détecté
    results[eps] = avg_period

    # Tracé de la dynamique de A pour la valeur courante de ε
    plt.plot(time_arr, A_vals, label=f'ε = {eps:.3f}')

plt.xlabel('Temps (unités)')
plt.ylabel('A (activateur)')
plt.title('Balayage paramétrique : effet de ε sur la dynamique de A')
plt.legend()
plt.grid(True)
plt.show()

print("Période moyenne (unités de temps) en fonction de ε :")
for eps, period in results.items():
    print(f"ε = {eps:.3f} : Période moyenne = {period:.3f}")

Période moyenne (unités de temps) en fonction de ε :
ε = 0.050 : Période moyenne = 44.251
ε = 0.088 : Période moyenne = 29.744
ε = 0.125 : Période moyenne = 23.304
ε = 0.163 : Période moyenne = 19.379
ε = 0.200 : Période moyenne = 16.978


/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/940056145.py:78: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Paramètres du modèle FHN
# -------------------------------
a = 0.5         # Intensité du terme de stimulation dans I_S (augmentée pour amplifier l'effet)
Kd = 5          # Constante de dissociation pour le cAMP
gamma = 0.15    # Facteur de couplage entre A et R
c0 = 2.1        # Terme constant influençant la récupération (ajusté pour fixer le point de repos)
epsilon = 0.088 # Facteur d'échelle pour la mise à jour de R
sigma = 0.1     # Amplitude du bruit (désactivé ici en pratique)

# -------------------------------
# Paramètres de production de cAMP
# -------------------------------
D = 2500.0      # Quantité de cAMP produite lorsque A dépasse le seuil
af = -1.2       # Seuil d'activation pour la production de cAMP

# -------------------------------
# Paramètres de la simulation
# -------------------------------
dt = 0.001       # Pas de temps
T_total = 100.0  # Temps total de simulation
n_steps = int(T_total / dt)

# Conditions initiales (point fixe de repos)
A = -2.0
R = 0.667

# Fonction pour calculer le terme d'excitation I_S à partir du signal externe de cAMP
def I_S(signal):
    return a * np.log1p(signal / Kd)

# -------------------------------
# Définition du signal externe de cAMP
# -------------------------------
signal = np.zeros(n_steps)
t_inject = 10.0         # Temps d'injection (en minutes, par exemple)
inject_duration = 5     # Durée de l'injection
inject_start = int(t_inject / dt)
inject_steps = int(inject_duration / dt)
signal[inject_start:inject_start + inject_steps] = 1000.0  # Injection d'un mini-pic de cAMP

# -------------------------------
# Stockage des valeurs pour tracer
# -------------------------------
A_vals = np.zeros(n_steps)
R_vals = np.zeros(n_steps)
production_vals = np.zeros(n_steps)  # Production de cAMP
time_arr = np.linspace(0, T_total, n_steps)

# -------------------------------
# Intégration numérique (méthode d'Euler)
# -------------------------------
for i in range(n_steps):
    A_vals[i] = A
    R_vals[i] = R
    current_I_S = I_S(signal[i])
    
    # Équations du modèle FHN
    dA = (A - (A ** 3) / 3 - R + current_I_S) * dt
    dR = epsilon * (A - gamma * R + c0) * dt
    
    # Mise à jour des états
    A += dA
    R += dR
    
    # Production de cAMP : la cellule produit D si A > af
    production_vals[i] = D if A > af else 0

# -------------------------------
# Tracé avec trois axes y distincts
# -------------------------------
fig, ax1 = plt.subplots(figsize=(10, 6))

# Axe 1 pour A et R
ax1.plot(time_arr, A_vals, label='A (activateur)', color='blue', lw=2)
ax1.plot(time_arr, R_vals, label='R (répresseur)', color='green', lw=2)
ax1.set_xlabel('Temps')
ax1.set_ylabel('A et R (valeurs)')
ax1.grid(True)

# Axe 2 pour la production de cAMP
ax2 = ax1.twinx()
ax2.plot(time_arr, production_vals, label='cAMP produit', color='red', lw=2, linestyle='--')
ax2.set_ylabel('cAMP produit (quantité)')

# Axe 3 pour le signal externe
ax3 = ax1.twinx()
ax3.spines["right"].set_position(("axes", 1.15))
ax3.plot(time_arr, signal, label='Signal externe', color='purple', lw=2, linestyle=':')
ax3.set_ylabel('Signal externe')
ax3.tick_params(axis='y', labelcolor='purple')

# Combinaison des légendes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
ax1.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, loc='upper right')

plt.title("Simulation FHN isolé : repos stable puis perturbation par signal externe")
plt.show()

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/25293455.py:103: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------------------------------------------
# Paramètres du modèle FitzHugh–Nagumo (FHN)
# -----------------------------------------------------------------
a = 0.1         # Intensité du terme de stimulation dans I_S
Kd = 5.0        # Constante de dissociation pour le cAMP
gamma = 0.15    # Facteur de couplage entre A et R
epsilon = 0.088  # Facteur d'échelle pour la mise à jour de R
af = 0.0        # Seuil d'activation pour la production de cAMP (exemple)

# On va faire varier c0 au cours du temps
c0_low = 2.1    # Valeur de c0 pour le régime initial (par ex. repos stable)
c0_high = 0.1   # Valeur de c0 pour le second régime (par ex. oscillatoire)
T_switch = 20.0 # Temps (en unités) où on bascule c0_low -> c0_high

# -----------------------------------------------------------------
# Paramètres de production de cAMP
# -----------------------------------------------------------------
D = 2500.0      # Quantité de cAMP produite lorsque A dépasse le seuil af

# -----------------------------------------------------------------
# Paramètres de simulation
# -----------------------------------------------------------------
dt = 0.01        # Pas de temps
T_total = 100.0  # Temps total
n_steps = int(T_total / dt)
time_arr = np.linspace(0, T_total, n_steps)

# -----------------------------------------------------------------
# Conditions initiales
# -----------------------------------------------------------------
A = -2.0
R = 0.67

# -----------------------------------------------------------------
# Définition (optionnelle) d'un signal externe
# -----------------------------------------------------------------
# Par exemple, un pic entre t=10 et t=15
signal = np.zeros(n_steps)
t_inject = 10.0
inject_duration = 5.0
inject_start = int(t_inject / dt)
inject_steps = int(inject_duration / dt)
signal[inject_start:inject_start + inject_steps] = 10.0  # Valeur 10 (ex.)

def I_S(signal_value):
    """
    Terme d'excitation en fonction du signal externe.
    On peut l'ajuster selon la forme souhaitée (ici, log1p).
    """
    return a * np.log1p(signal_value / Kd)

def c0_temps(t):
    """
    Fonction c0(t) : renvoie c0_low si t < T_switch, sinon c0_high.
    On peut aussi faire une transition douce (logistique, linéaire, etc.).
    """
    return c0_low if t < T_switch else c0_high

# -----------------------------------------------------------------
# Stockage des trajectoires
# -----------------------------------------------------------------
A_vals = np.zeros(n_steps)
R_vals = np.zeros(n_steps)
production_vals = np.zeros(n_steps)

# -----------------------------------------------------------------
# Boucle d'intégration (Euler explicite)
# -----------------------------------------------------------------
for i in range(n_steps):
    A_vals[i] = A
    R_vals[i] = R

    # Valeur du signal externe à l'instant courant
    current_signal = signal[i]
    # Calcul du terme d'excitation
    current_I_S = I_S(current_signal)

    # Récupération de c0(t)
    current_c0 = c0_temps(time_arr[i])

    # Equations FHN
    dA = (A - (A**3)/3 - R + current_I_S) * dt
    dR = epsilon * (A - gamma*R + current_c0) * dt

    # Mise à jour
    A += dA
    R += dR

    # Production de cAMP si A > af
    production_vals[i] = D if (A > af) else 0

# -----------------------------------------------------------------
# Tracé
# -----------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(10, 6))

# A et R
ax1.plot(time_arr, A_vals, label="A (activateur)", color='blue')
ax1.plot(time_arr, R_vals, label="R (répresseur)", color='green')
ax1.set_xlabel("Temps")
ax1.set_ylabel("A et R")
ax1.grid(True)

# Production de cAMP
ax2 = ax1.twinx()
ax2.plot(time_arr, production_vals, label="cAMP produit", color='red', linestyle='--')
ax2.set_ylabel("Production cAMP")

# Légendes combinées
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.title("FHN avec variation de c0(t) : transition d'un régime à l'autre")
plt.show()

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/478838832.py:118: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------------------------------------------
# Paramètres du modèle FitzHugh–Nagumo (FHN)
# -----------------------------------------------------------------
a = 0.1         # Intensité du terme de stimulation dans I_S
Kd = 5.0        # Constante de dissociation pour le cAMP
gamma = 0.15    # Facteur de couplage entre A et R
epsilon = 0.08  # Facteur d'échelle pour la mise à jour de R
af = 0.0        # Seuil d'activation pour la production de cAMP (exemple)

# On va faire varier c0 au cours du temps
c0_low = 2.1    # Valeur de c0 pour le régime initial (par ex. repos stable)
c0_high = 0.1   # Valeur de c0 pour le second régime (par ex. oscillatoire)
T_switch = 20.0 # Temps (en unités) où on bascule c0_low -> c0_high

# -----------------------------------------------------------------
# Paramètres de production de cAMP
# -----------------------------------------------------------------
D = 2500.0      # Quantité de cAMP produite lorsque A dépasse le seuil af

# -----------------------------------------------------------------
# Paramètres de simulation
# -----------------------------------------------------------------
dt = 0.01        # Pas de temps
T_total = 100.0  # Temps total
n_steps = int(T_total / dt)
time_arr = np.linspace(0, T_total, n_steps)

# -----------------------------------------------------------------
# Conditions initiales
# -----------------------------------------------------------------
A = -2.0
R = 0.67

# -----------------------------------------------------------------
# Définition (optionnelle) d'un signal externe
# -----------------------------------------------------------------
# Par exemple, un pic entre t=10 et t=15
signal = np.zeros(n_steps)
t_inject = 10.0
inject_duration = 5.0
inject_start = int(t_inject / dt)
inject_steps = int(inject_duration / dt)
signal[inject_start:inject_start + inject_steps] = 10.0  # Valeur 10 (ex.)

def I_S(signal_value):
    """
    Terme d'excitation en fonction du signal externe.
    On peut l'ajuster selon la forme souhaitée (ici, log1p).
    """
    return a * np.log1p(signal_value / Kd)

def c0_temps(t):
    """
    Fonction c0(t) : renvoie c0_low si t < T_switch, sinon c0_high.
    On peut aussi faire une transition douce (logistique, linéaire, etc.).
    """
    return c0_low if t < T_switch else c0_high

# -----------------------------------------------------------------
# Stockage des trajectoires
# -----------------------------------------------------------------
A_vals = np.zeros(n_steps)
R_vals = np.zeros(n_steps)
production_vals = np.zeros(n_steps)

# -----------------------------------------------------------------
# Boucle d'intégration (Euler explicite)
# -----------------------------------------------------------------
for i in range(n_steps):
    A_vals[i] = A
    R_vals[i] = R

    # Valeur du signal externe à l'instant courant
    current_signal = signal[i]
    # Calcul du terme d'excitation
    current_I_S = I_S(current_signal)

    # Récupération de c0(t)
    current_c0 = c0_temps(time_arr[i])

    # Equations FHN
    dA = (A - (A**3)/3 - R + current_I_S) * dt
    dR = epsilon * (A - gamma*R + current_c0) * dt

    # Mise à jour
    A += dA
    R += dR

    # Production de cAMP si A > af
    production_vals[i] = D if (A > af) else 0

# -----------------------------------------------------------------
# Tracé
# -----------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(10, 6))

# A et R
ax1.plot(time_arr, A_vals, label="A (activateur)", color='blue')
ax1.plot(time_arr, R_vals, label="R (répresseur)", color='green')
ax1.set_xlabel("Temps")
ax1.set_ylabel("A et R")
ax1.grid(True)

# Production de cAMP
ax2 = ax1.twinx()
ax2.plot(time_arr, production_vals, label="cAMP produit", color='red', linestyle='--')
ax2.set_ylabel("Production cAMP")

# Légendes combinées
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.title("FHN avec variation de c0(t) : transition d'un régime à l'autre")
plt.show()

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/2221700164.py:118: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Exemple de simulation couplant :
 - Un champ de cAMP (diffusion + dégradation) en 2D,
 - Une cellule unique décrite par un modèle FitzHugh–Nagumo,
   dont le paramètre c0 est remplacé par la concentration locale de cAMP.
 
La cellule peut ainsi rester au repos (si la concentration locale en cAMP est faible)
et passer en régime oscillatoire (si la concentration locale augmente suffisamment).
 
Le code enregistre et trace également les oscillations internes de la cellule (A, R).
 
Auteur : ChatGPT
Date   : 2025-02-13
"""

import numpy as np
import matplotlib.pyplot as plt
import csv
import sys

# =============================================================================
# Paramètres globaux
# =============================================================================
SPACE_SIZE = 50        # Taille d'un côté de la grille 2D (en µm, par ex.)
GRID_RESOLUTION = 1.0  # Résolution spatiale (taille d'une case, en µm)
DT = 0.1               # Pas de temps
T_MAX = 200            # Temps total de simulation
NB_STEPS = int(T_MAX / DT)

# =============================================================================
# Paramètres du champ de cAMP
# =============================================================================
D_CAMP = 2.0    # Coefficient de diffusion du cAMP (µm²/min)
PDE_RATE = 0.1  # Taux de dégradation du cAMP
PRODUCTION_BASAL = 0.0  # Production basale (si vous en avez besoin)
THRESH_ACTIV = 0.5      # Seuil arbitraire pour un éventuel couplage (optionnel)

# =============================================================================
# Paramètres FitzHugh–Nagumo (pour la cellule)
# =============================================================================
A_INIT = -1.5      # Condition initiale pour A
R_INIT = 0.0       # Condition initiale pour R
EPSILON = 0.05     # Facteur d'échelle (dR/dt)
GAMMA = 0.5        # Facteur de couplage R
A_COEFF = 0.1      # Facteur pour I_S = A_COEFF * log1p( cAMP / Kd )
KD = 0.5           # Constante de dissociation pour le cAMP (terme I_S)
D_PROD = 10.0      # Quantité de cAMP produite quand A dépasse un certain seuil
A_THRESHOLD = 0.5  # Seuil d'activation interne de la cellule (si A > A_THRESHOLD -> production)

# =============================================================================
# Classe CampField : gère la diffusion + dégradation du cAMP en 2D
# =============================================================================
class CampField:
    """
    Représente un champ de cAMP sur une grille 2D.
    Diffusion par différences finies (Laplace) et dégradation linéaire.
    """
    def __init__(self, space_size, grid_res, diff_coeff, degrade_rate):
        """
        Paramètres
        ----------
        space_size : float
            Taille du domaine 2D (carré).
        grid_res : float
            Résolution spatiale (taille d'une cellule de grille).
        diff_coeff : float
            Coefficient de diffusion du cAMP.
        degrade_rate : float
            Taux de dégradation (PDE).
        """
        self.space_size = space_size
        self.grid_res = grid_res
        self.diff_coeff = diff_coeff
        self.degrade_rate = degrade_rate
        
        # Nombre de points dans la grille
        self.grid_size = int(self.space_size / self.grid_res)
        
        # Tableau 2D pour la concentration de cAMP
        self.camp = np.zeros((self.grid_size, self.grid_size), dtype=float)
        
        # Calcul du pas pour la diffusion
        self.dx = self.grid_res
        
    def laplacian(self, mat):
        """
        Calcule le laplacien 2D par différences finies,
        avec conditions aux bords périodiques.
        """
        # On peut utiliser np.roll pour un voisinage périodique
        lap = (np.roll(mat, 1, axis=0) + np.roll(mat, -1, axis=0)
               + np.roll(mat, 1, axis=1) + np.roll(mat, -1, axis=1)
               - 4.0 * mat) / (self.dx ** 2)
        return lap
    
    def update(self, dt):
        """
        Met à jour la concentration de cAMP selon :
          dC/dt = D_camp * Laplacien(C) - PDE_rate * C
        """
        lap = self.laplacian(self.camp)
        dC = self.diff_coeff * lap - self.degrade_rate * self.camp
        self.camp += dC * dt
        
        # Clamp pour éviter des valeurs négatives
        np.clip(self.camp, 0.0, None, out=self.camp)
    
    def add_camp(self, x_idx, y_idx, amount):
        """
        Ajoute 'amount' de cAMP à la position (x_idx, y_idx) dans la grille.
        """
        self.camp[x_idx, y_idx] += amount
    
    def get_local_camp(self, x_idx, y_idx):
        """
        Retourne la concentration locale de cAMP à (x_idx, y_idx).
        """
        return self.camp[x_idx, y_idx]

# =============================================================================
# Classe CellAgent : représente la cellule unique (FHN)
# =============================================================================
class CellAgent:
    """
    Modèle FitzHugh–Nagumo pour une cellule.
    c0 est remplacé par la concentration locale en cAMP (local_camp).
    """
    def __init__(self, A_init, R_init, epsilon, gamma, a_coeff, kd, 
                 a_threshold, d_prod, x_idx, y_idx):
        """
        Paramètres
        ----------
        A_init : float
            Condition initiale pour A.
        R_init : float
            Condition initiale pour R.
        epsilon : float
            Facteur d'échelle dans l'équation de R.
        gamma : float
            Facteur de couplage R.
        a_coeff : float
            Coefficient pour I_S = a_coeff * log1p(camp/kd).
        kd : float
            Constante de dissociation pour le log(1 + c/kd).
        a_threshold : float
            Seuil d'activation interne de A (si A > a_threshold -> production).
        d_prod : float
            Quantité de cAMP produite quand A > a_threshold.
        x_idx, y_idx : int
            Position de la cellule sur la grille.
        """
        self.A = A_init
        self.R = R_init
        self.epsilon = epsilon
        self.gamma = gamma
        self.a_coeff = a_coeff
        self.kd = kd
        self.a_threshold = a_threshold
        self.d_prod = d_prod
        
        self.x_idx = x_idx
        self.y_idx = y_idx
        
    def update_state(self, local_camp, dt):
        """
        Met à jour les variables A, R en fonction de la concentration locale de cAMP
        (qui remplace c0 dans l'équation de R).
        
        dA/dt = A - A^3/3 - R + I_S
        dR/dt = epsilon * (A - gamma*R + local_camp)
        
        où I_S = a_coeff * log1p(local_camp / kd).
        
        Retourne la quantité de cAMP produite par la cellule à cet instant.
        """
        # Terme d'excitation
        I_S = self.a_coeff * np.log1p(local_camp / self.kd)
        
        # Équations FHN
        dA = (self.A - (self.A**3)/3 - self.R + I_S) * dt
        dR = self.epsilon * (self.A - self.gamma * self.R + local_camp) * dt
        
        self.A += dA
        self.R += dR
        
        # Production de cAMP si A > seuil
        if self.A > self.a_threshold:
            return self.d_prod
        else:
            return 0.0

# =============================================================================
# Programme principal
# =============================================================================
def main():
    """
    Lance la simulation :
     - Initialise un champ de cAMP (50x50).
     - Place une cellule unique au centre.
     - À chaque itération, met à jour le champ de cAMP puis l'état FHN de la cellule.
     - Stocke l'évolution de A, R dans un fichier CSV et trace le résultat.
    """
    # Création du champ de cAMP
    camp_field = CampField(SPACE_SIZE, GRID_RESOLUTION, D_CAMP, PDE_RATE)
    grid_size = camp_field.grid_size
    
    # Position de la cellule au centre
    x_cell = grid_size // 2
    y_cell = grid_size // 2
    
    # Création de la cellule FHN
    cell = CellAgent(A_init=A_INIT,
                     R_init=R_INIT,
                     epsilon=EPSILON,
                     gamma=GAMMA,
                     a_coeff=A_COEFF,
                     kd=KD,
                     a_threshold=A_THRESHOLD,
                     d_prod=D_PROD,
                     x_idx=x_cell,
                     y_idx=y_cell)
    
    # Tableaux pour enregistrer les données (A, R, cAMP local)
    A_list = []
    R_list = []
    camp_local_list = []
    time_list = []
    
    # Boucle de simulation
    for step in range(NB_STEPS):
        t = step * DT
        
        # 1) Mise à jour du champ de cAMP (diffusion + dégradation)
        camp_field.update(DT)
        
        # 2) Lecture de la concentration locale en cAMP
        local_camp = camp_field.get_local_camp(cell.x_idx, cell.y_idx)
        
        # 3) Mise à jour de la cellule FHN avec local_camp
        produced = cell.update_state(local_camp, DT)
        
        # 4) Injection de cAMP produit dans le champ
        if produced > 0:
            camp_field.add_camp(cell.x_idx, cell.y_idx, produced)
        
        # Stockage
        A_list.append(cell.A)
        R_list.append(cell.R)
        camp_local_list.append(local_camp)
        time_list.append(t)
    
    # Enregistrement dans un fichier CSV
    with open("single_cell_oscillation.csv", "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["time", "A", "R", "local_cAMP"])
        for i in range(NB_STEPS):
            writer.writerow([time_list[i], A_list[i], R_list[i], camp_local_list[i]])
    
    # Tracé des oscillations internes (A, R) et de la concentration locale en cAMP
    fig, ax1 = plt.subplots(figsize=(8, 6))
    
    ax1.plot(time_list, A_list, label="A (activateur)", color='blue')
    ax1.plot(time_list, R_list, label="R (répresseur)", color='green')
    ax1.set_xlabel("Temps (min)")
    ax1.set_ylabel("A et R")
    ax1.grid(True)
    
    ax2 = ax1.twinx()
    ax2.plot(time_list, camp_local_list, label="cAMP local", color='red', linestyle='--')
    ax2.set_ylabel("Concentration locale en cAMP")
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='best')
    
    plt.title("Oscillations d'une cellule FHN couplée au champ de cAMP (c0 = cAMP local)")
    plt.tight_layout()
    plt.savefig("single_cell_oscillation.png", dpi=200)
    plt.show()

if __name__ == "__main__":
    # Pour éviter les éventuels problèmes d'encodage sous Windows
    try:
        main()
    except KeyboardInterrupt:
        print("Simulation interrompue par l'utilisateur.")
        sys.exit(0)

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/1844108977.py:281: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Simulation de la propagation d'AMPc dans une population de cellules Dictyostelium.

Ce script simule l'évolution d'un champ de signal (AMPc) diffusif couplé à une dynamique 
cellulaire. Des images du champ de signal sont sauvegardées périodiquement dans le dossier 
output_files.

Les fonctions sont optimisées avec Numba pour le CPU et les matrices creuses de SciPy sont 
utilisées pour la diffusion.
"""

from __future__ import division
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Permet de sauvegarder les figures sans affichage interactif
import matplotlib.pyplot as plt
import os
import scipy.sparse as sparse
from numba import njit

# =============================================================================
# Fonctions de signalisation et utilitaires
# =============================================================================

@njit
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    """
    Calcule l'incrément du signal.

    Parameters:
        dt (float): pas de temps.
        lap (ndarray): laplacien du signal.
        Dsig (float): coefficient de diffusion du signal.
        J (float): coefficient de réaction (aPDE).
        rho (float): terme de production.
        a0 (float): constante de production.
        D (float): coefficient supplémentaire dans le terme source.
        signal (ndarray): champ de signal actuel.
        A_grid_hsided (ndarray): grille issue de l'état des agents (après application d'une fonction seuil).

    Returns:
        ndarray: incrément du signal.
    """
    return dt * (Dsig * lap - J * signal + rho * a0) + dt * (rho * D * A_grid_hsided)


def accumulate_arr(coord, arr, shape):
    """
    Accumule les valeurs d'un tableau d'agents dans une grille 2D.

    Cette fonction utilise les coordonnées des agents pour sommer leurs valeurs
    dans une grille de dimension 'shape'.

    Parameters:
        coord (ndarray): tableau de dimensions (2, nombre_d_agents) contenant les coordonnées.
        arr (ndarray): tableau des valeurs associées aux agents.
        shape (tuple): dimensions (n_lignes, n_colonnes) de la grille de sortie.

    Returns:
        ndarray: grille 2D avec les sommes accumulées.
    """
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0] * shape[1]).reshape(shape)


def scale(A, B, k):
    """
    Remplit la matrice A par la matrice B en créant des blocs de taille k x k.

    Parameters:
        A (ndarray): matrice de destination.
        B (ndarray): matrice source.
        k (int): facteur d'échelle (taille du bloc).

    Returns:
        ndarray: matrice A modifiée.
    """
    Y, X = A.shape
    for y in range(k):
        for x in range(k):
            A[y:Y:k, x:X:k] = B
    return A


def downscale_array(arr, scale_factor):
    """
    Réduit la taille d'un tableau 'arr' par un facteur 'scale_factor'
    en faisant la moyenne sur des blocs.

    Parameters:
        arr (ndarray): tableau 2D à réduire.
        scale_factor (int): facteur de réduction.

    Returns:
        ndarray: tableau réduit en moyenne.
    """
    new_shape = (arr.shape[0] // scale_factor, scale_factor,
                 arr.shape[1] // scale_factor, scale_factor)
    return arr.reshape(new_shape).mean(axis=(1, 3))


def calc_square_laplacian_noflux_matrix(size):
    """
    Construit la matrice laplacienne 2D avec conditions de bord « no flux »
    pour une grille de dimensions 'size'.

    Parameters:
        size (tuple): dimensions (n_lignes, n_colonnes) de la grille.

    Returns:
        csr_matrix: matrice laplacienne sous format compressé.
    """
    nrows, ncols = size
    # Matrice identité pour la direction x
    Ix = sparse.eye(ncols)
    # Vecteur de 1 pour construire la matrice de différences en x
    e_x = np.ones(ncols)
    Ax = sparse.diags([-e_x, 2 * e_x, -e_x], [-1, 0, 1], shape=(ncols, ncols))
    # Matrice identité pour la direction y
    Iy = sparse.eye(nrows)
    e_y = np.ones(nrows)
    Ay = sparse.diags([-e_y, 2 * e_y, -e_y], [-1, 0, 1], shape=(nrows, nrows))
    # Produit de Kronecker pour obtenir le laplacien 2D
    lap = sparse.kron(Ay, Ix) + sparse.kron(Iy, Ax)
    return lap.tocsr()


# =============================================================================
# Définition des paramètres par défaut
# =============================================================================

# Paramètres de la grille
grid_params_default = {
    'dx': 0.5,               # pas spatial
    'D_sig': 5,              # coefficient de diffusion du signal
    'box_size_x': 50,        # taille en x (unités)
    'box_size_y': 50,        # taille en y (unités)
    'agent_dim': 1,          # dimension caractéristique de l'agent (exemple : 1)
    'num_agents': 15000      # nombre total d'agents
}

# Paramètres cellulaires
cell_params_default = {
    'c0': 1,
    'a': 0.05,
    'gamma': 0.5,
    'Kd': 1e-6,
    'sigma': 0.15,
    'epsilon': 0.2,
    'cStim': 100,
    'aPDE': 10,
    'rho': 1,
    'D': 1000,
    'a0': 1,
    'af': 0
}



In [14]:

# =============================================================================
# Classe de simulation de la population
# =============================================================================

class DictyPop:
    """
    Classe pour simuler la propagation d'AMPc et la dynamique cellulaire
    dans une population de cellules Dictyostelium.

    Attributes:
        g_params (dict): paramètres de la grille.
        c_params (dict): paramètres cellulaires.
        T (float): temps total de simulation.
        dt (float): pas de temps.
        Tsteps (int): nombre de pas de simulation.
        signal_size (tuple): dimensions de la grille de signal.
        fluxes (ndarray): termes de flux aux bords.
        A (ndarray): état A de chaque agent.
        R (ndarray): état R de chaque agent.
        signal (ndarray): champ de signal (AMPc).
        ...
    """

    def __init__(self, T, save_every, g_params=grid_params_default,
                 c_params=cell_params_default, noise=True, progress_report=True,
                 save_data=True):
        """
        Initialise la simulation.

        Parameters:
            T (float): temps total de simulation.
            save_every (float): intervalle de sauvegarde en temps.
            g_params (dict): paramètres de la grille.
            c_params (dict): paramètres cellulaires.
            noise (bool): si True, ajoute du bruit à la dynamique.
            progress_report (bool): si True, affiche l'avancement.
            save_data (bool): si True, sauvegarde l'évolution.
        """
        self.g_params = g_params
        self.c_params = c_params
        self.T = T            # Temps total de simulation (unités de temps)
        self.ts = 0           # Temps courant
        # Choix du pas de temps (dépend de dx et de D_sig)
        self.dt = self.g_params['dx']**2 / (8 * self.g_params['D_sig'])
        self.Tsteps = int(self.T / self.dt)
        # Taille de la grille de signal
        self.signal_size = (int(self.g_params['box_size_x'] / self.g_params['dx']),
                            int(self.g_params['box_size_y'] / self.g_params['dx']))
        self.fluxes = np.zeros(4)  # Pour d'éventuels flux sur les bords
        self.progress_report = progress_report
        self.save_every = save_every
        self.save_data = save_data

        # Rapport entre la taille d'agent et le pas spatial (pour downscaling)
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])
        if self.agent_ratio < 1:
            self.agent_ratio = 1
        # Grille pour les agents et grille "agrandie" pour interpolation
        self.A_grid = np.zeros((self.signal_size[0] // self.agent_ratio,
                                self.signal_size[1] // self.agent_ratio))
        self.A_grid_big = np.zeros(self.signal_size)
        # Initialisation du champ de signal (AMPc)
        self.signal = np.zeros(self.signal_size)
        self.noise_flag = noise

        # Initialisation de l'état cellulaire (deux composantes : A et R)
        num_agents = self.g_params['num_agents']
        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, num_agents))
        self.A = cell_state[0, :]
        self.R = cell_state[1, :]

        self.D = float(self.c_params['D'])
        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)

        # Initialisation des coordonnées (sur la grille réduite) de chaque agent
        grid_shape = self.A_grid.shape
        self.coord = np.zeros((2, num_agents), dtype=int)
        for agent in range(num_agents):
            self.coord[0, agent] = np.random.randint(0, grid_shape[0])
            self.coord[1, agent] = np.random.randint(0, grid_shape[1])

        # Construction de la matrice laplacienne 2D avec conditions de bord « no flux »
        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            # Cas 1D (rare)
            n = max(self.signal_size)
            self.lap_mat = sparse.diags([np.ones(n-1), -2 * np.ones(n), np.ones(n-1)],
                                        [-1, 0, 1], format="csr")
            self.lap_mat += sparse.diags(np.array([1] + [0]*(n-2) + [1]), format="csr")

        # Sauvegarde de l'évolution si demandé
        if self.save_data:
            num_save = int(self.T // self.save_every) + 1
            self.A_saved = np.zeros((self.A_grid.shape[0], self.A_grid.shape[1], num_save))
            self.R_saved = np.zeros((self.A_grid.shape[0], self.A_grid.shape[1], num_save))
            self.S_saved = np.zeros((self.signal.shape[0], self.signal.shape[1], num_save))
            self.A_saved[:, :, 0] = self.getAGrid()
            self.R_saved[:, :, 0] = self.getRGrid()
            self.S_saved[:, :, 0] = self.signal

    def getAGrid(self):
        """
        Accumule l'état A des agents dans la grille réduite.

        Returns:
            ndarray: grille 2D des valeurs de A.
        """
        return accumulate_arr(self.coord, self.A, self.A_grid.shape)

    def getRGrid(self):
        """
        Accumule l'état R des agents dans la grille réduite.

        Returns:
            ndarray: grille 2D des valeurs de R.
        """
        return accumulate_arr(self.coord, self.R, self.A_grid.shape)

    def setSignal(self, signal):
        """
        Définit le champ de signal.

        Parameters:
            signal (ndarray): nouvelle grille de signal.
        """
        assert signal.shape == self.signal.shape and signal.dtype == self.signal.dtype, \
            "Input signal incorrect shape or type"
        self.signal = np.array(signal)

    def setCellState(self, state):
        """
        Définit l'état cellulaire pour tous les agents.

        Parameters:
            state (ndarray): tableau de forme (2, nombre_d_agents) contenant les états.
        """
        assert state.shape[0] == 2 and state.shape[1] == self.A.shape[0], \
            "Input cell state incorrect shape or type"
        self.A = np.array(state[0, :])
        self.R = np.array(state[1, :])

    def setFluxes(self, fluxes):
        """
        Définit les flux appliqués aux bords du domaine.

        Parameters:
            fluxes (ndarray): tableau de forme (4,) contenant les flux sur chaque bord.
        """
        assert fluxes.shape == self.fluxes.shape and fluxes.dtype == self.fluxes.dtype, \
            "Input fluxes incorrect shape or type"
        self.fluxes = np.array(fluxes)

    def getdA(self):
        """
        Calcule la dérivée de A pour chaque agent.

        La dérivée de A est calculée en fonction de la dynamique interne 
        (effet auto-activant avec saturation et inhibition par R) et d'une 
        contribution dépendant du signal local.

        Pour éviter une erreur dans np.log1p, on s'assure que l'argument ne soit 
        pas inférieur à -0.999.

        Returns:
            ndarray: incrément de A pour chaque agent.
        """
        # Réduction du champ de signal vers la grille des agents (moyenne sur blocs)
        agent_signal = downscale_array(self.signal, self.agent_ratio)
        # Récupération de la valeur locale du signal pour chaque agent
        local_signal = agent_signal[self.coord[0, :], self.coord[1, :]]
        # Calcul du ratio en évitant les valeurs trop négatives (pour log1p)
        ratio = local_signal / self.c_params['Kd']
        print(ratio)
        ratio = np.minimum(ratio, 1e4)  # Par exemple, on limite le ratio à 1e6
        ratio = np.maximum(ratio, -0.999)
        print("2", ratio)

        return ((self.A - (self.A**3) / 3 - self.R) +
                (self.c_params['a'] * np.log1p(ratio))) * self.dt

    def getdR(self):
        """
        Calcule la dérivée de R pour chaque agent.

        Returns:
            ndarray: incrément de R pour chaque agent.
        """
        return ((self.A - self.c_params['gamma'] * self.R + self.c_params['c0'])
                * self.c_params['epsilon']) * self.dt

    def getLapC(self):
        """
        Calcule la laplacienne du champ de signal.

        Returns:
            ndarray: laplacien du signal sous forme d'une grille 2D.
        """
        flat_signal = self.signal.flatten()
        laplacian = (1 / (self.g_params['dx']**2)) * self.lap_mat.dot(flat_signal)
        return laplacian.reshape(self.signal.shape)

    def update(self):
        """
        Effectue une mise à jour de l'état des agents et du champ de signal.

        Cette méthode :
          - Met à jour les états A et R des agents.
          - Met à jour le champ de signal à l'aide d'une équation de diffusion-réaction.
          - Applique des conditions aux bords.

        Returns:
            ndarray: nouveau champ de signal.
        """
        self.ts += self.dt
        self.dA = self.getdA()
        if self.noise_flag:
            self.dA += self.c_params['sigma'] * np.random.normal(
                loc=0.0, scale=np.sqrt(self.dt), size=self.A.shape)
        self.dR = self.getdR()

        # Mise à jour de la grille associée aux agents
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        self.dsignal = addcD(self.dt, self.getLapC(), self.g_params['D_sig'],
                             self.c_params['aPDE'], self.c_params['rho'],
                             self.c_params['a0'], self.c_params['D'],
                             self.signal, np.heaviside(self.A_grid_big, 0.5))

        # Mise à jour des états et du champ de signal
        self.A += self.dA
        self.R += self.dR
        self.signal += self.dsignal

        # Application des conditions aux bords (flux)
        dx = self.g_params['dx']
        self.signal[0, 1:-1] -= self.fluxes[0] / dx
        self.signal[-1, 1:-1] -= self.fluxes[1] / dx
        self.signal[1:-1, 0] -= self.fluxes[2] / dx
        self.signal[1:-1, -1] -= self.fluxes[3] / dx
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * dx)
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * dx)
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * dx)
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * dx)
        return self.signal


# =============================================================================
# Fonction principale de simulation
# =============================================================================

def run_simulation():
    """
    Exécute la simulation de propagation d'AMPc et sauvegarde périodiquement les images du signal.

    La simulation s'exécute sur un temps total T et les images sont sauvegardées dans le dossier 'output_files'.
    """
    # Paramètres de la simulation
    T = 10              # Temps total de simulation (unités arbitraires)
    save_every = 1      # Sauvegarde des données toutes les 1 unité de temps
    sim = DictyPop(T, save_every)
    steps = sim.Tsteps

    output_dir = "output_files"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Début de la simulation sur {steps} étapes (dt = {sim.dt:.4f}) ...")
    for step in range(steps):
        S = sim.update()
        # Sauvegarde d'une image tous les 100 pas de temps
        if step % 100 == 0:
            plt.figure()
            plt.imshow(S, cmap='viridis', origin='lower')
            plt.title(f"t = {sim.ts:.2f}")
            plt.colorbar(label="Signal")
            filename = os.path.join(output_dir, f"signal_{step:05d}.png")
            plt.savefig(filename)
            plt.close()
            print(f"Étape {step}/{steps} – image sauvegardée : {filename}")
    print("Simulation terminée.")


# =============================================================================
# Lancement de la simulation
# =============================================================================

if __name__ == "__main__":
    run_simulation()

Début de la simulation sur 1600 étapes (dt = 0.0063) ...
[0. 0. 0. ... 0. 0. 0.]
2 [0. 0. 0. ... 0. 0. 0.]
Étape 0/1600 – image sauvegardée : output_files/signal_00000.png
[6.25625e+06 6.25000e+03 6.25625e+06 ... 6.25625e+06 6.25000e+03
 6.25625e+06]
2 [10000.  6250. 10000. ... 10000.  6250. 10000.]
[12512109.375  -378515.625 12902734.375 ... 12902734.375 -1159765.625
 12512109.375]
2 [ 1.00e+04 -9.99e-01  1.00e+04 ...  1.00e+04 -9.99e-01  1.00e+04]
[18840844.7265625 -1251928.7109375 20085961.9140625 ... 20110375.9765625
 -3839819.3359375 18889672.8515625]
2 [ 1.00e+04 -9.99e-01  1.00e+04 ...  1.00e+04 -9.99e-01  1.00e+04]
[25337083.4350586  -2761976.62353516 27998222.35107422 ...
 28115708.92333985 -8523695.37353516 25556809.9975586 ]
2 [ 1.00e+04 -9.99e-01  1.00e+04 ...  1.00e+04 -9.99e-01  1.00e+04]
[ 32129497.62344361  -5522041.22543335  36898092.74673462 ...
  37257976.62734985 -15914420.98617554  32748622.98965454]
2 [ 1.00e+04 -9.99e-01  1.00e+04 ...  1.00e+04 -9.99e-01  1.00e+0

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_16906/1019101991.py:175: RuntimeWarning: overflow encountered in divide
  ratio = local_signal / self.c_params['Kd']


Étape 1100/1600 – image sauvegardée : output_files/signal_01100.png
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[nan nan nan ... nan nan nan]
2 [nan nan nan ... nan nan nan]
[n

SyntaxError: invalid character '’' (U+2019) (906437557.py, line 27)

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# @author: Alexander Golden

### libraries import ###
import numpy as np
import matplotlib
import random as rand
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import math as math
import os
import sys
import scipy.stats as sc
import csv
import pandas as pd
import torch
import time
import sys
from __future__ import division
import scipy as sp
import scipy.sparse as sparse
# import line_profiler
from numba import njit, float64
from matplotlib.colors import LogNorm
import itertools as it
import random as rand
sys.argv = ['']
import pickle as pkl
import argparse

### 'signaling functions' ###
@njit(float64[:,:](float64,float64[:,:],float64,float64,float64,float64,float64,float64[:,:],float64[:,:]))
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    return dt*(Dsig*lap - J*signal + rho*a0) + dt*(rho*D*A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    # Get linear indices to be used as IDs with bincount
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0]*shape[1]).reshape(shape[0], shape[1])

def scale(A, B, k):
    # fill A with B scaled by k
    Y = A.shape[0]
    X = A.shape[1]
    for y in range(0, k):
        for x in range(0, k):
            A[y:Y:k, x:X:k] = B
    return A

def scale1D(A, B, k):
    # fill A with B scaled by k
    Y = A.shape[0]
    for y in range(0, k):
        A[y:Y:k] = B
    return A

def scale_down_mat_sp(current_shape, scale):
    """
    Generates a matrix that takes a matrix of one size and scales it down by
    a factor given by integer argument "scale" by first flattening the
    original matrix.
    Output is a matrix with dimensions (m,n) where m = total size of scaled
    array and n = total size of original array.
    Make sure the desired scale is consistent with the matrix shape.
    """
    assert current_shape[0]//scale == current_shape[0]/scale, "Shape not divisible by scale"
    assert current_shape[1]//scale == current_shape[1]/scale, "Shape not divisible by scale"

    # Define the shape of the scaled down matrix
    out_shape = (current_shape[0]//scale, current_shape[1]//scale)

    # Define top row of the scale-down matrix
    line = np.array(([1/scale**2]*scale + [0]*(current_shape[0]-scale))*scale
                    + [0]*(current_shape[0]*(current_shape[1]-scale)))

    # Initialize the output matrix starting with this row
    out_mat = sp.sparse.bsr_matrix(line)

    # Generate and add rows to deal with the entire first row of the scaled-down matrix
    for i in range(out_shape[0]-1):
        newline = np.roll(line, scale*(i+1))
        out_mat = sp.sparse.vstack([out_mat, newline])

    # Block is a matrix that scales down an entire line of the original matrix
    block = out_mat.todense()

    # Generate the rest of the matrix by shifting and stacking copies of block
    for i in range(out_shape[1]-1):
        newblock = sp.sparse.bsr_matrix(np.roll(block, scale*current_shape[0]*(i+1), 1))
        out_mat = sp.sparse.vstack([out_mat, newblock])

    return sparse.csr_matrix(out_mat)

def calc_square_laplacian_noflux_matrix(size):
    # set up basic laplacian block diagonal matrix
    xDiff = np.zeros((size[1]+1, size[1]))
    ix, jx = np.indices(xDiff.shape)
    xDiff[ix == jx] = 1
    xDiff[ix == jx+1] = -1

    yDiff = np.zeros((size[0]+1, size[0]))
    iy, jy = np.indices(yDiff.shape)
    yDiff[iy == jy] = 1
    yDiff[iy == jy+1] = -1

    Ax = sparse.dia_matrix(-np.matmul(np.transpose(xDiff), xDiff))
    Ay = sparse.dia_matrix(-np.matmul(np.transpose(yDiff), yDiff))

    lap = sparse.kron(Ay, sparse.eye(size[1]), format='csr') + \
          sparse.kron(sparse.eye(size[0]), Ax, format='csr')

    # set up boundary conditions
    lap += sparse.diags([2] + [1]*(size[1]-2) + [2] +
                        ([1] + [0]*(size[1]-2) + [1])*(size[0]-2) +
                        [2] + [1]*(size[1]-2) + [2])
    lap = sparse.bsr_matrix(lap)
    return lap

def lapMatGivenFlux(size, fluxes):
    return False

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

### Population of signaling cells ###
class dictyPop:
    def __init__(self, T, save_every,
                 g_params={'dx': 0.5, 'D_sig': 5, 'box_size_x': 50, 'box_size_y': 50,
                           'agent_dim': 1, 'num_agents': 100, 'num_agents_1': 0},
                 c_params={'c0': 1, 'a': 0.05, 'gamma': 0.5, 'Kd': 1e-5, 'sigma': 0.15,
                           'epsilon': 0.2, 'cStim': 100, 'aPDE': 10, 'rho': 1,
                           'D': 1000, 'a0': 1, 'af': 0},
                 noise=True, progress_report=True, save_data=True):
        self.g_params = g_params
        self.c_params = c_params
        self.T = T
        self.ts = 0
        self.dt = self.g_params['dx']**2/(8*self.g_params['D_sig'])
        self.Tsteps = int(self.T/self.dt)

        self.signal_size = (
            int(self.g_params['box_size_x']/self.g_params['dx']),
            int(self.g_params['box_size_y']/self.g_params['dx'])
        )

        self.fluxes = np.zeros(4)
        self.p_r = progress_report
        self.save_every = save_every
        self.save_data = save_data

        self.agent_ratio = int(self.g_params['agent_dim']/self.g_params['dx'])
        self.agent_size = (
            max(1, int(self.g_params['box_size_x']/self.g_params['agent_dim'])),
            max(1, int(self.g_params['box_size_y']/self.g_params['agent_dim']))
        )

        self.signal = np.zeros(self.signal_size)
        self.noise_flag = noise

        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, g_params['num_agents']))
        self.A = cell_state[0, :]
        self.R = cell_state[1, :]

        # agent_1 = rand.sample(range(0, self.g_params['num_agents']), self.g_params['num_agents_1'])
        self.D = float(self.c_params['D'])

        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)

        self.interp_mat = scale_down_mat_sp(self.signal_size, self.agent_ratio)
        self.A_grid = np.zeros((self.signal_size[0]//self.agent_ratio, self.signal_size[1]//self.agent_ratio))
        self.A_grid_big = np.zeros(self.signal.shape)

        self.coord = np.zeros((2, self.g_params['num_agents']), dtype='int')
        # Exemple d'initialisation aléatoire (remplacer ll, agent_dim, dx par vos variables si besoin)
        for agent in range(self.g_params['num_agents']):
            # Supposons que 'll' et 'dx' soient définis ou remplacés par une borne
            # Forcément, adaptez à vos besoins
            # Ici, on met une borne arbitraire de 50
            ll = 50
            self.coord[:, agent] = np.array([
                int(rand.random()*ll/max(self.g_params['agent_dim'], self.g_params['dx'])),
                int(rand.random()*ll/max(self.g_params['agent_dim'], self.g_params['dx']))
            ])

        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            self.lap_mat = sparse.diags(
                [np.ones(max(self.signal_size)-1),
                 -2*np.ones(max(self.signal_size)),
                 np.ones(max(self.signal_size)-1)],
                [-1, 0, 1],
                format='csr'
            )
            self.lap_mat += sparse.diags(
                np.array([1] + [0]*(max(self.signal_size)-2) + [1]),
                format='csr'
            )

        if self.save_data is True:
            steps_count = (self.T // save_every) + 1
            self.A_saved = np.zeros((self.signal_size[0]//self.agent_ratio,
                                     self.signal_size[1]//self.agent_ratio,
                                     steps_count))
            self.R_saved = np.zeros((self.signal_size[0]//self.agent_ratio,
                                     self.signal_size[1]//self.agent_ratio,
                                     steps_count))
            self.S_saved = np.zeros((self.signal_size[0],
                                     self.signal_size[1],
                                     steps_count))

            self.A_saved[:, :, 0] = self.getAGrid()
            self.R_saved[:, :, 0] = self.getRGrid()
            self.S_saved[:, :, 0] = self.signal

    def getAGrid(self):
        return accumulate_arr(self.coord, self.A, self.A_grid.shape)

    def getRGrid(self):
        return accumulate_arr(self.coord, self.R, self.A_grid.shape)

    def setSignal(self, signal):
        assert (self.signal.shape == signal.shape) and (self.signal.dtype == signal.dtype), "Input signal incorrect shape or type"
        self.signal = np.array(signal)

    def setCellState(self, state):
        assert (self.A.shape == state[:, 0].shape) and (self.A.dtype == state.dtype), "Input cell state incorrect shape or type"
        self.A = np.array(state[:, 0])
        self.R = np.array(state[:, 1])

    def setFluxes(self, fluxes):
        assert (self.fluxes.shape == fluxes.shape) and (self.fluxes.dtype == fluxes.dtype), "Input fluxes incorrect shape or type"
        self.fluxes = np.array(fluxes)

    def getdA(self):
        return ((self.A - (self.A*self.A*self.A)/3 - self.R)) * self.dt + \
               (self.c_params['a'] * np.log1p(
                   np.reshape(
                       self.interp_mat.dot(
                           np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])
                       ),
                       self.agent_size
                   )[self.coord[0, :], self.coord[1, :]] / self.c_params['Kd']
               )) * self.dt

    def getdR(self):
        return ((self.A - self.c_params['gamma']*self.R) + self.c_params['c0']) * \
               self.c_params['epsilon'] * self.dt

    def getdC(self):
        # get laplacian for 0 flux
        laplacian = 1/(self.g_params['dx']**2) * np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1])
        )

        # set fluxes
        self.signal[0, 1:-1] -= self.fluxes[0]/self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1]/self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2]/self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3]/self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3])/(2*self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3])/(2*self.g_params['dx'])

        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)

        return self.dt*(
            self.g_params['D_sig']*laplacian
            - self.c_params['aPDE']*self.signal
            + self.c_params['rho']*self.c_params['a0']
        ) + self.dt*(
            self.c_params['rho']*self.c_params['D'] * np.heaviside(self.A_grid_big, 0.5)
        )

    def getLapC(self):
        return 1/(self.g_params['dx']**2) * np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1])
        )

    def update(self):
        self.ts += self.dt
        self.dA = self.getdA()

        if self.noise_flag:
            self.dA += self.c_params['sigma'] * np.random.normal(
                loc=0.0,
                scale=np.sqrt(self.dt),
                size=self.A.shape
            )

        self.dR = self.getdR()

        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)

        self.dsignal = addcD(
            self.dt,
            self.getLapC(),
            self.g_params['D_sig'],
            self.c_params['aPDE'],
            self.c_params['rho'],
            self.c_params['a0'],
            self.c_params['D'],
            self.signal,
            np.heaviside(self.A_grid_big, 0.5)
        )

        self.A += self.dA
        self.R += self.dR
        self.signal += self.dsignal

        # fluxes again after updating
        self.signal[0, 1:-1] -= self.fluxes[0]/self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1]/self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2]/self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3]/self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3])/(2*self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3])/(2*self.g_params['dx'])

        return self.signal

### Run the script and produce timeseries of signal intensity ###
save = 100
size = 10  # size of the area from which timeseries are extracted

def timeseries_a(a, t0, tf, freq):
    #### Cell and grid parameter ###
    path = 'output_files/'
    print(path)
    N = int(15000)  # Population size

    cell_params_default = {
        'c0': 1, 'a': 0.05, 'gamma': 0.5, 'Kd': 10**(-5),
        'sigma': 0.15, 'epsilon': 0.2, 'cStim': 100, 'aPDE': 10,
        'rho': 1, 'D': 1000, 'a0': 1, 'af': 0
    }

    # Remarque: dx n'est pas défini localement dans cette fonction
    # On peut imaginer qu'il est défini dans cell_params_default ou plus haut
    dx = 0.5

    grid_params_default = {
        'dx': dx, 'D_sig': 5, 'box_size_x': 50, 'box_size_y': 50,
        'agent_dim': 2*dx, 'num_agents': N, 'num_agents_1': 0
    }

    # paramètre 'params' non défini dans cette fonction, à clarifier :
    # on utilise cell_params_default à la place ou on suppose que 'params' = cell_params_default ?
    params = cell_params_default

    noise = True
    pop = dictyPop(1, 1, c_params=params, g_params=grid_params_default, noise=noise)

    pop.A = np.zeros(pop.A.shape)
    pop.R = np.zeros(pop.R.shape)

    S = pop.update()

    TimeSeries = np.array([(np.nan,) for i in range(int((len(S)/size)**2))])

    for i in range(t0, tf):
        S = pop.update()
        NewCol = []
        if i % 100 == 0:
            plt.matshow(S)
            plt.title('t=' + str(i))
            plt.savefig(str(i))
            plt.show()

        if i % save == 0:
            for ii in range(0, len(S), size):
                for jj in range(0, len(S), size):
                    NewCol.append(np.mean(S[ii:ii+size, jj:jj+size]) / np.mean(S))
            NCol = pd.DataFrame({'col': NewCol})
            TimeSeries = np.column_stack((TimeSeries, NCol))

    return TimeSeries

In [8]:
# Exemple d'appel (attention aux paramètres)
result = timeseries_a(a=0.15, t0=0, tf=1000, freq=10)

output_files/


/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_20614/1201262.py:379: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Simulation de signalisation dans une population de Dictyostelium.

Ce script définit :
  - Des fonctions utilitaires pour le traitement de matrices et le calcul du laplacien.
  - La classe DictyPop qui modélise la population de cellules signalantes.
  - Une fonction timeseries_a qui lance la simulation et sauvegarde une série temporelle du signal.

Paramètres par défaut (d'après Table S1) :
  - N (nombre d'agents) = 15000
  - l (box_size_x, box_size_y) = 50
  - dx = 0.5
  - a = 0.05
  - Kd (K) = 1e-5
  - sigma = 0.15
  - epsilon = 0.2
  - cStim = 100
  - aPDE = 10
  - rho = 1
  - a0 = 1
  - gamma = 0.5
  - c0 = 1
  - af = 0
  - D = 1000
  - Dsig = 5
"""

# ========================
# IMPORTATIONS
# ========================
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Backend 'Agg' pour sauvegarder les figures sans affichage interactif
import matplotlib.pyplot as plt
import os
import sys
import scipy as sp
import scipy.sparse as sparse
import pandas as pd
import random as rand
from numba import njit, float64

# Pour éviter certains conflits d'arguments avec notebooks
sys.argv = ['']

# ========================
# FONCTIONS UTILITAIRES
# ========================

@njit(float64[:, :](float64, float64[:, :], float64, float64, float64, float64, float64, float64[:, :], float64[:, :]))
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    """
    Calcule la variation du signal (dC) sur un pas de temps dt.

    dC = dt * (Dsig * lap - J * signal + rho * a0)
         + dt * (rho * D * A_grid_hsided)
    """
    return dt * (Dsig * lap - J * signal + rho * a0) + dt * (rho * D * A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    """
    Accumule les valeurs de 'arr' dans une matrice de dimensions 'shape'
    selon des indices donnés par 'coord'.
    """
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0] * shape[1]).reshape(shape)

def scale(A, B, k):
    """
    Remplit la matrice A avec la matrice B répétée sur une grille de facteur k.
    """
    Y, X = A.shape
    for y in range(k):
        for x in range(k):
            A[y:Y:k, x:X:k] = B
    return A

def scale1D(A, B, k):
    """
    Version 1D de la fonction scale : répète le vecteur B dans A par bloc de taille k.
    """
    Y = A.shape[0]
    for y in range(k):
        A[y:Y:k] = B
    return A

def scale_down_mat_sp(current_shape, scale):
    """
    Génère une matrice de réduction (downscaling) sous forme creuse.

    Cette matrice permet de réduire une matrice de taille 'current_shape'
    par un facteur 'scale' en réalisant une moyenne sur des blocs.
    """
    assert current_shape[0] % scale == 0, "La dimension 0 n'est pas divisible par scale"
    assert current_shape[1] % scale == 0, "La dimension 1 n'est pas divisible par scale"

    out_shape = (current_shape[0] // scale, current_shape[1] // scale)

    # Création de la première ligne du filtre
    line = np.array(([1 / scale**2] * scale + [0] * (current_shape[0] - scale)) * scale +
                    [0] * (current_shape[0] * (current_shape[1] - scale)))
    out_mat = sparse.bsr_matrix(line)

    # Génération des lignes décalées
    for i in range(out_shape[0] - 1):
        newline = np.roll(line, scale * (i + 1))
        out_mat = sparse.vstack([out_mat, newline])

    block = out_mat.todense()

    # Construction du filtre complet par décalage horizontal
    for i in range(out_shape[1] - 1):
        newblock = sparse.bsr_matrix(np.roll(block, scale * current_shape[0] * (i + 1), axis=1))
        out_mat = sparse.vstack([out_mat, newblock])

    return sparse.csr_matrix(out_mat)

def calc_square_laplacian_noflux_matrix(size):
    """
    Calcule la matrice laplacienne 2D pour une grille de taille 'size'
    avec conditions aux limites de type pas-flux.
    """
    xDiff = np.zeros((size[1] + 1, size[1]))
    ix, jx = np.indices(xDiff.shape)
    xDiff[ix == jx] = 1
    xDiff[ix == jx + 1] = -1

    yDiff = np.zeros((size[0] + 1, size[0]))
    iy, jy = np.indices(yDiff.shape)
    yDiff[iy == jy] = 1
    yDiff[iy == jy + 1] = -1

    Ax = sparse.dia_matrix(-np.matmul(xDiff.T, xDiff))
    Ay = sparse.dia_matrix(-np.matmul(yDiff.T, yDiff))

    lap = sparse.kron(Ay, sparse.eye(size[1]), format='csr') + \
          sparse.kron(sparse.eye(size[0]), Ax, format='csr')

    # Ajout des conditions aux limites (modification de la diagonale)
    lap += sparse.diags([2] + [1] * (size[1] - 2) + [2] +
                        ([1] + [0] * (size[1] - 2) + [1]) * (size[0] - 2) +
                        [2] + [1] * (size[1] - 2) + [2])
    return sparse.bsr_matrix(lap)

def lapMatGivenFlux(size, fluxes):
    """
    Fonction non implémentée pour la prise en compte de flux imposés.
    """
    return False

# ========================
# CLASSE : DictyPop
# ========================
class DictyPop:
    """
    Classe représentant une population de cellules signalantes.

    Cette classe gère la dynamique de deux variables cellulaires (A et R)
    ainsi que la diffusion et la réaction du signal sur une grille.
    """
    def __init__(self, T, save_every,
                 g_params=None,
                 c_params=None,
                 noise=True, 
                 progress_report=True, 
                 save_data=True):
        """
        Initialise la simulation.

        Paramètres:
          - T : temps total de la simulation.
          - save_every : fréquence de sauvegarde.
          - g_params : dictionnaire des paramètres de la grille.
          - c_params : dictionnaire des paramètres cellulaires.
          - noise : active (True) ou désactive le bruit.
          - progress_report : active le suivi de progression.
          - save_data : active la sauvegarde des données.
        """
        # -----------------
        # Paramètres grille
        # -----------------
        if g_params is None:
            g_params = {
                'dx': 0.5,
                'D_sig': 5,
                'box_size_x': 50,
                'box_size_y': 50,
                'agent_dim': 1,
                'num_agents': 15000,
                'num_agents_1': 0
            }
        self.g_params = g_params

        # -----------------
        # Paramètres cellules
        # -----------------
        if c_params is None:
            c_params = {
                'c0': 1,
                'a': 0.05,
                'gamma': 0.5,
                'Kd': 1e-5,
                'sigma': 0.15,
                'epsilon': 0.2,
                'cStim': 100,
                'aPDE': 10,
                'rho': 1,
                'D': 1000,
                'a0': 1,
                'af': 0
            }
        self.c_params = c_params

        self.T = T
        self.ts = 0  # Temps actuel
        self.dt = self.g_params['dx']**2 / (8 * self.g_params['D_sig'])
        self.Tsteps = int(self.T / self.dt)

        # Taille de la grille du signal
        self.signal_size = (
            int(self.g_params['box_size_x'] / self.g_params['dx']),
            int(self.g_params['box_size_y'] / self.g_params['dx'])
        )

        self.fluxes = np.zeros(4)
        self.p_r = progress_report
        self.save_every = save_every
        self.save_data = save_data

        # Paramètres liés aux agents
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])
        self.agent_size = (
            max(1, int(self.g_params['box_size_x'] / self.g_params['agent_dim'])),
            max(1, int(self.g_params['box_size_y'] / self.g_params['agent_dim']))
        )

        # Initialisation du signal et des variables cellulaires
        self.signal = np.zeros(self.signal_size)
        self.noise_flag = noise

        # État initial aléatoire de A et R
        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, self.g_params['num_agents']))
        self.A = cell_state[0, :]
        self.R = cell_state[1, :]

        # Coefficient D
        self.D = float(self.c_params['D'])

        # Variables pour stocker les dérivées
        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)

        # Matrice d'interpolation pour passer de la grille de signal à la grille d'agents
        self.interp_mat = scale_down_mat_sp(self.signal_size, self.agent_ratio)
        self.A_grid = np.zeros((self.signal_size[0] // self.agent_ratio,
                                self.signal_size[1] // self.agent_ratio))
        self.A_grid_big = np.zeros(self.signal.shape)

        # Initialisation aléatoire des coordonnées des agents
        self.coord = np.zeros((2, self.g_params['num_agents']), dtype='int')
        for agent in range(self.g_params['num_agents']):
            # On suppose une borne spatiale 'l' = 50
            ll = 50
            self.coord[:, agent] = np.array([
                int(rand.random() * ll / max(self.g_params['agent_dim'], self.g_params['dx'])),
                int(rand.random() * ll / max(self.g_params['agent_dim'], self.g_params['dx']))
            ])

        # Calcul de la matrice laplacienne avec conditions pas-flux
        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            # Cas unidimensionnel
            size1D = max(self.signal_size)
            self.lap_mat = sparse.diags(
                [np.ones(size1D - 1), -2 * np.ones(size1D), np.ones(size1D - 1)],
                [-1, 0, 1],
                format='csr'
            )
            self.lap_mat += sparse.diags(
                np.array([1] + [0] * (size1D - 2) + [1]),
                format='csr'
            )

        # Initialisation des tableaux de sauvegarde (si nécessaire)
        if self.save_data:
            steps_count = (self.T // save_every) + 1
            self.A_saved = np.zeros((self.signal_size[0] // self.agent_ratio,
                                     self.signal_size[1] // self.agent_ratio,
                                     steps_count))
            self.R_saved = np.zeros((self.signal_size[0] // self.agent_ratio,
                                     self.signal_size[1] // self.agent_ratio,
                                     steps_count))
            self.S_saved = np.zeros((self.signal_size[0],
                                     self.signal_size[1],
                                     steps_count))

            self.A_saved[:, :, 0] = self.getAGrid()
            self.R_saved[:, :, 0] = self.getRGrid()
            self.S_saved[:, :, 0] = self.signal

    # -----------------
    # Méthodes de grille
    # -----------------
    def getAGrid(self):
        """ Accumule les valeurs de A dans la grille A_grid. """
        return accumulate_arr(self.coord, self.A, self.A_grid.shape)

    def getRGrid(self):
        """ Accumule les valeurs de R dans la grille A_grid. """
        return accumulate_arr(self.coord, self.R, self.A_grid.shape)

    # -----------------
    # Méthodes de mise à jour
    # -----------------
    def setSignal(self, signal):
        """ Met à jour la matrice du signal. """
        assert (self.signal.shape == signal.shape) and (self.signal.dtype == signal.dtype), \
            "Signal d'entrée incorrect"
        self.signal = np.array(signal)

    def setCellState(self, state):
        """ Met à jour les états cellulaires A et R. """
        assert (self.A.shape == state[:, 0].shape) and (self.A.dtype == state.dtype), \
            "Etat cellulaire d'entrée incorrect"
        self.A = np.array(state[:, 0])
        self.R = np.array(state[:, 1])

    def setFluxes(self, fluxes):
        """ Met à jour les flux appliqués sur les bords du signal. """
        assert (self.fluxes.shape == fluxes.shape) and (self.fluxes.dtype == fluxes.dtype), \
            "Flux d'entrée incorrect"
        self.fluxes = np.array(fluxes)

    # -----------------
    # Équations d'évolution
    # -----------------
    def getdA(self):
        """
        Calcule la variation de A pour un pas de temps dt.
        """
        # Dynamique locale (FHN-like)
        dA_local = (self.A - (self.A ** 3) / 3 - self.R) * self.dt

        # Contribution liée au signal interpolé
        interp_signal = np.reshape(
            self.interp_mat.dot(self.signal.ravel()),
            self.agent_size
        )
        dA_signal = (self.c_params['a'] *
                     np.log1p(interp_signal[self.coord[0, :], self.coord[1, :]] / self.c_params['Kd'])
                     ) * self.dt

        return dA_local + dA_signal

    def getdR(self):
        """
        Calcule la variation de R pour un pas de temps dt.
        """
        return ((self.A - self.c_params['gamma'] * self.R) + self.c_params['c0']) * \
               self.c_params['epsilon'] * self.dt

    def getLapC(self):
        """
        Calcule le laplacien du signal.
        """
        return (1 / (self.g_params['dx'] ** 2)) * np.reshape(
            self.lap_mat.dot(self.signal.ravel()),
            self.signal_size
        )

    def update(self):
        """
        Met à jour la simulation sur un pas de temps dt.
        Retourne la nouvelle matrice du signal.
        """
        self.ts += self.dt

        # Variation de A et ajout de bruit si nécessaire
        self.dA = self.getdA()
        if self.noise_flag:
            self.dA += self.c_params['sigma'] * np.random.normal(
                loc=0.0, scale=np.sqrt(self.dt), size=self.A.shape
            )
        # Variation de R
        self.dR = self.getdR()

        # Mise à jour de A et R
        self.A += self.dA
        self.R += self.dR

        # Mise à jour des grilles d'agents
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)

        # Variation du signal
        lapC = self.getLapC()
        self.dsignal = addcD(
            self.dt,
            lapC,
            self.g_params['D_sig'],
            self.c_params['aPDE'],
            self.c_params['rho'],
            self.c_params['a0'],
            self.c_params['D'],
            self.signal,
            np.heaviside(self.A_grid_big, 0.5)
        )

        self.signal += self.dsignal

        # Réapplication des flux aux bords
        self.signal[0, 1:-1] -= self.fluxes[0] / self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1] / self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2] / self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3] / self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * self.g_params['dx'])

        return self.signal

# ========================
# FONCTION : timeseries_a
# ========================
def timeseries_a(a, t0, tf, freq):
    """
    Lance la simulation et génère une série temporelle de l'intensité du signal.

    Paramètres:
      - a : paramètre d'activation (non indispensable ici si on fixe c_params).
      - t0 : itération de départ.
      - tf : itération finale.
      - freq : fréquence d'enregistrement (non utilisée directement).

    Retourne:
      - Un tableau numpy contenant la série temporelle des valeurs moyennes du signal.
    """
    # Répertoire de sortie pour les figures
    output_path = 'output_files/'
    os.makedirs(output_path, exist_ok=True)
    print("Répertoire de sortie :", output_path)

    # -----------------
    # Paramètres par défaut (Table S1)
    # -----------------
    N = 15000
    box_size = 50
    dx = 0.5
    c_params_default = {
        'c0': 1,
        'a': 0.05,
        'gamma': 0.5,
        'Kd': 1e-5,      # K
        'sigma': 0.15,
        'epsilon': 0.2,
        'cStim': 100,
        'aPDE': 10,
        'rho': 1,
        'D': 1000,
        'a0': 1,
        'af': 0
    }
    g_params_default = {
        'dx': dx,
        'D_sig': 5,
        'box_size_x': box_size,
        'box_size_y': box_size,
        'agent_dim': 1,      # Vous pouvez changer si nécessaire (ex. 2*dx)
        'num_agents': N,
        'num_agents_1': 0
    }

    # Initialisation de la population
    noise = True
    pop = DictyPop(
        T=1,
        save_every=1,
        c_params=c_params_default,
        g_params=g_params_default,
        noise=noise
    )

    # Initialisation de A et R à zéro pour la simulation
    pop.A = np.zeros(pop.A.shape)
    pop.R = np.zeros(pop.R.shape)

    # Première mise à jour
    S = pop.update()

    # Initialisation de la série temporelle (remplie d'abord de NaN)
    size = 10  # Taille du bloc pour le calcul des moyennes locales
    TimeSeries = np.array([(np.nan,) for _ in range(int((len(S) / size) ** 2))])

    # Boucle de simulation
    for i in range(t0, tf):
        S = pop.update()
        NewCol = []

        # Sauvegarde d'une figure tous les 100 pas
        if i % 100 == 0:
            plt.matshow(S)
            plt.title(f"t = {i}")
            plt.savefig(os.path.join(output_path, f"{i}.png"))
            plt.close()

        # Enregistrement périodique des moyennes locales du signal
        if i % 100 == 0:
            for ii in range(0, len(S), size):
                for jj in range(0, len(S), size):
                    block_mean = np.mean(S[ii:ii + size, jj:jj + size])
                    NewCol.append(block_mean / np.mean(S))
            TimeSeries = np.column_stack((TimeSeries, pd.DataFrame({'col': NewCol})))

    return TimeSeries

# ========================
# EXEMPLE D'EXECUTION
# ========================
if __name__ == "__main__":
    # Exemple : simulation de 1000 pas de temps
    ts = timeseries_a(a=0.05, t0=0, tf=1000, freq=10)

Répertoire de sortie : output_files/


/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_20614/2454604478.py:506: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.matshow(S)


In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Simulation de la propagation d'AMPc et de la dynamique cellulaire dans une population 
de cellules Dictyostelium.

Ce script simule l'évolution d'un champ de signal diffusif couplé à la dynamique 
cellulaire (modélisée par un système de type FitzHugh–Nagumo) et sauvegarde périodiquement 
des images du champ de signal dans le dossier 'output_files'. 
De plus, il enregistre et trace les valeurs de l'état A et R d'une cellule (ici, la première cellule)
en fonction du temps.

Les fonctions sont optimisées avec Numba et les matrices creuses de SciPy sont utilisées pour la diffusion.
"""

# =============================================================================
# IMPORTATIONS
# =============================================================================
from __future__ import division
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Permet de sauvegarder les figures sans affichage interactif
import matplotlib.pyplot as plt
import os
import sys
import scipy as sp
import scipy.sparse as sparse
import pandas as pd
import random as rand
from numba import njit, float64

# On vide sys.argv pour éviter des interférences éventuelles
sys.argv = ['']

# =============================================================================
# FONCTIONS UTILITAIRES ET DE SIGNALISATION
# =============================================================================

@njit(float64[:, :](float64, float64[:, :], float64, float64, float64, float64, float64, float64[:, :], float64[:, :]))
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    """
    Calcule l'incrément du signal sur un pas de temps dt en combinant diffusion et réaction.
    
    Formule utilisée :
        dC = dt * (Dsig * lap - J * signal + rho * a0)
             + dt * (rho * D * A_grid_hsided)
    
    Parameters:
        dt (float): pas de temps.
        lap (ndarray): laplacien du signal.
        Dsig (float): coefficient de diffusion du signal.
        J (float): coefficient de réaction (aPDE).
        rho (float): taux de production.
        a0 (float): terme constant de production.
        D (float): coefficient additionnel modulant la contribution des agents.
        signal (ndarray): champ de signal actuel.
        A_grid_hsided (ndarray): grille obtenue après application d'une fonction seuil sur la répartition des agents.
    
    Returns:
        ndarray: incrément du signal.
    """
    return dt * (Dsig * lap - J * signal + rho * a0) + dt * (rho * D * A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    """
    Accumule les valeurs d'un tableau (arr) aux positions indiquées par 'coord' dans une grille de dimensions 'shape'.
    
    Parameters:
        coord (ndarray): tableau de dimensions (2, nombre_d_agents) contenant les coordonnées des agents.
        arr (ndarray): tableau des valeurs associées à chaque agent.
        shape (tuple): dimensions (n_lignes, n_colonnes) de la grille de sortie.
    
    Returns:
        ndarray: grille 2D avec les sommes accumulées.
    """
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0] * shape[1]).reshape(shape)

def scale(A, B, k):
    """
    Remplit la matrice A par des copies de la matrice B placées dans des blocs de taille k x k.
    
    Parameters:
        A (ndarray): matrice de destination.
        B (ndarray): matrice source.
        k (int): facteur d'échelle (taille du bloc).
    
    Returns:
        ndarray: matrice A modifiée.
    """
    Y, X = A.shape
    for y in range(k):
        for x in range(k):
            A[y:Y:k, x:X:k] = B
    return A

def downscale_array(arr, scale_factor):
    """
    Réduit la taille d'un tableau 2D en faisant la moyenne sur des blocs de taille scale_factor x scale_factor.
    
    Parameters:
        arr (ndarray): tableau 2D à réduire.
        scale_factor (int): facteur de réduction.
    
    Returns:
        ndarray: tableau réduit par moyenne sur blocs.
    """
    new_shape = (arr.shape[0] // scale_factor, scale_factor,
                 arr.shape[1] // scale_factor, scale_factor)
    return arr.reshape(new_shape).mean(axis=(1, 3))

def calc_square_laplacian_noflux_matrix(size):
    """
    Construit la matrice laplacienne 2D pour une grille de dimensions 'size' avec conditions aux bords "no flux".
    
    Parameters:
        size (tuple): dimensions (n_lignes, n_colonnes) de la grille.
    
    Returns:
        csr_matrix: matrice laplacienne sous format compressé.
    """
    # Création des matrices de différences pour la direction x
    xDiff = np.zeros((size[1] + 1, size[1]))
    ix, jx = np.indices(xDiff.shape)
    xDiff[ix == jx] = 1
    xDiff[ix == jx + 1] = -1

    # Création des matrices de différences pour la direction y
    yDiff = np.zeros((size[0] + 1, size[0]))
    iy, jy = np.indices(yDiff.shape)
    yDiff[iy == jy] = 1
    yDiff[iy == jy + 1] = -1

    # Calcul des produits matriciels pour obtenir le laplacien en x et y
    Ax = sparse.dia_matrix(-np.matmul(xDiff.T, xDiff))
    Ay = sparse.dia_matrix(-np.matmul(yDiff.T, yDiff))

    # Construction du laplacien 2D via le produit de Kronecker
    lap = sparse.kron(Ay, sparse.eye(size[1]), format='csr') + \
          sparse.kron(sparse.eye(size[0]), Ax, format='csr')

    # Ajustement de la diagonale pour prendre en compte les conditions aux bords
    lap += sparse.diags([2] + [1]*(size[1]-2) + [2] +
                        ([1] + [0]*(size[1]-2) + [1])*(size[0]-2) +
                        [2] + [1]*(size[1]-2) + [2])
    return sparse.bsr_matrix(lap)

# =============================================================================
# DÉFINITION DES PARAMÈTRES PAR DÉFAUT
# =============================================================================

# Paramètres de la grille
grid_params_default = {
    'dx': 0.5,               # Pas spatial : distance entre deux points consécutifs de la grille (par exemple en microns)
    'D_sig': 5,              # Coefficient de diffusion du signal : module la vitesse de propagation du signal
    'box_size_x': 50,        # Taille totale de la grille en x : largeur du domaine de simulation
    'box_size_y': 50,        # Taille totale de la grille en y : hauteur du domaine de simulation
    'agent_dim': 1,          # Dimension caractéristique d'un agent : utilisée pour le downscaling et la répartition des agents
    'num_agents': 15000      # Nombre total d'agents (cellules) dans la simulation
}

# Paramètres cellulaires
cell_params_default = {
    'c0': 0.1,               # Terme constant c0 : valeur de base dans la dynamique cellulaire
    'a': 0.11,               # Coefficient d'activation : module la réponse d'activation des cellules au signal
    'gamma': 0.5,            # Coefficient d'inhibition : module la rétroaction inhibitrice sur l'activation
    'Kd': 1e-5,              # Constante de dissociation (Kd) : détermine la sensibilité de la réponse cellulaire au signal
    'sigma': 0.5,            # Amplitude du bruit : introduit des fluctuations stochastiques dans la dynamique cellulaire
    'epsilon': 0.2,          # Paramètre d'échelle temporelle pour la variable R : ajuste la vitesse de la réponse inhibitrice
    'cStim': 10,             # Niveau de stimulation externe : intensité du stimulus appliqué aux cellules
    'aPDE': 100,             # Coefficient dans l'équation de réaction-diffusion : module la dégradation ou l'interaction entre le signal et la cellule
    'rho': 0.1,              # Taux de production du signal : détermine la quantité de signal produite par les cellules
    'D': 10,                 # Coefficient additionnel : module la contribution de l'état des agents à la production du signal
    'a0': 1,                 # Terme constant dans la production du signal : valeur de base de production
    'af': 0                  # Paramètre additionnel (souvent mis à zéro) pour d'éventuelles influences supplémentaires
}

# =============================================================================
# CLASSE DE SIMULATION : DictyPop
# =============================================================================

class DictyPop:
    """
    Classe pour simuler la propagation d'AMPc et la dynamique cellulaire dans une population 
    de cellules Dictyostelium.

    Attributes:
        g_params (dict): paramètres de la grille.
        c_params (dict): paramètres cellulaires.
        T (float): temps total de simulation.
        dt (float): pas de temps utilisé pour la simulation.
        Tsteps (int): nombre total de pas de simulation.
        signal_size (tuple): dimensions de la grille du signal.
        fluxes (ndarray): termes de flux aux bords.
        A (ndarray): état A de chaque agent.
        R (ndarray): état R de chaque agent.
        signal (ndarray): champ de signal (AMPc).
    """
    
    def __init__(self, T, save_every, g_params=grid_params_default,
                 c_params=cell_params_default, noise=True, progress_report=True,
                 save_data=True):
        """
        Initialise la simulation avec les paramètres donnés.
        
        Parameters:
            T (float): temps total de simulation (unités de temps).
            save_every (float): intervalle de sauvegarde en temps.
            g_params (dict, optionnel): paramètres de la grille (par défaut grid_params_default).
            c_params (dict, optionnel): paramètres cellulaires (par défaut cell_params_default).
            noise (bool, optionnel): si True, ajoute du bruit à la dynamique cellulaire.
            progress_report (bool, optionnel): si True, affiche l'avancement de la simulation.
            save_data (bool, optionnel): si True, sauvegarde l'évolution des états (non utilisé ici pour le tracé).
        """
        self.g_params = g_params
        self.c_params = c_params
        self.T = T            # Temps total de simulation
        self.ts = 0           # Temps courant initialisé à zéro
        # Calcul du pas de temps basé sur la condition de stabilité pour la diffusion
        self.dt = self.g_params['dx']**2 / (8 * self.g_params['D_sig'])
        self.Tsteps = int(self.T / self.dt)
        
        # Détermination des dimensions de la grille de signal
        self.signal_size = (int(self.g_params['box_size_x'] / self.g_params['dx']),
                            int(self.g_params['box_size_y'] / self.g_params['dx']))
        self.fluxes = np.zeros(4)  # Initialisation des flux aux bords (pour d'éventuels ajustements)
        self.progress_report = progress_report
        self.save_every = save_every
        self.save_data = save_data
        
        # Rapport entre la taille d'un agent et le pas spatial (pour downscaling)
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])
        if self.agent_ratio < 1:
            self.agent_ratio = 1
        # Création de la grille pour les agents (version réduite) et une grille agrandie pour l'interpolation
        self.A_grid = np.zeros((self.signal_size[0] // self.agent_ratio,
                                self.signal_size[1] // self.agent_ratio))
        self.A_grid_big = np.zeros(self.signal_size)
        # Initialisation du champ de signal (AMPc) à zéro
        self.signal = np.zeros(self.signal_size)
        self.noise_flag = noise
        
        # Initialisation de l'état cellulaire (deux composantes : A et R)
        num_agents = self.g_params['num_agents']
        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, num_agents))
        self.A = cell_state[0, :]
        self.R = cell_state[1, :]
        
        # Coefficient D utilisé dans la production du signal
        self.D = float(self.c_params['D'])
        # Initialisation des dérivées pour A, R et le signal
        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)
        
        # Attribution aléatoire des coordonnées des agents dans la grille réduite
        grid_shape = self.A_grid.shape
        self.coord = np.zeros((2, num_agents), dtype=int)
        for agent in range(num_agents):
            self.coord[0, agent] = np.random.randint(0, grid_shape[0])
            self.coord[1, agent] = np.random.randint(0, grid_shape[1])
        
        # Construction de la matrice laplacienne 2D avec conditions "no flux"
        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            # Cas 1D (rare)
            n = max(self.signal_size)
            self.lap_mat = sparse.diags([np.ones(n-1), -2 * np.ones(n), np.ones(n-1)],
                                        [-1, 0, 1], format="csr")
            self.lap_mat += sparse.diags(np.array([1] + [0]*(n-2) + [1]), format="csr")
        
        # Sauvegarde de l'évolution (tableaux pour A, R et le signal) si nécessaire
        if self.save_data:
            num_save = int(self.T // self.save_every) + 1
            self.A_saved = np.zeros((self.A_grid.shape[0], self.A_grid.shape[1], num_save))
            self.R_saved = np.zeros((self.A_grid.shape[0], self.A_grid.shape[1], num_save))
            self.S_saved = np.zeros((self.signal.shape[0], self.signal.shape[1], num_save))
            self.A_saved[:, :, 0] = self.getAGrid()
            self.R_saved[:, :, 0] = self.getRGrid()
            self.S_saved[:, :, 0] = self.signal

    def getAGrid(self):
        """
        Accumule l'état A des agents dans la grille réduite.
        
        Returns:
            ndarray: grille 2D des valeurs de A.
        """
        safe_A = np.nan_to_num(self.A, nan=0.0)
        return accumulate_arr(self.coord, safe_A, self.A_grid.shape)

    def getRGrid(self):
        """
        Accumule l'état R des agents dans la grille réduite.
        
        Returns:
            ndarray: grille 2D des valeurs de R.
        """
        safe_R = np.nan_to_num(self.R, nan=0.0)
        return accumulate_arr(self.coord, safe_R, self.A_grid.shape)

    def setSignal(self, signal):
        """
        Définit le champ de signal.
        
        Parameters:
            signal (ndarray): nouvelle grille de signal.
        """
        assert signal.shape == self.signal.shape and signal.dtype == self.signal.dtype, \
            "Input signal incorrect shape or type"
        self.signal = np.array(signal)

    def setCellState(self, state):
        """
        Définit l'état cellulaire pour tous les agents.
        
        Parameters:
            state (ndarray): tableau de forme (2, nombre_d_agents) contenant les états A et R.
        """
        assert state.shape[0] == 2 and state.shape[1] == self.A.shape[0], \
            "Input cell state incorrect shape or type"
        self.A = np.array(state[0, :])
        self.R = np.array(state[1, :])

    def setFluxes(self, fluxes):
        """
        Définit les flux appliqués aux bords du domaine.
        
        Parameters:
            fluxes (ndarray): tableau de forme (4,) contenant les flux sur chaque bord.
        """
        assert fluxes.shape == self.fluxes.shape and fluxes.dtype == self.fluxes.dtype, \
            "Input fluxes incorrect shape or type"
        self.fluxes = np.array(fluxes)

    def getdA(self):
        """
        Calcule la dérivée de A pour chaque agent.
        
        La dérivée de A est basée sur une dynamique de type FitzHugh–Nagumo avec 
        une contribution liée au signal local.
        
        Returns:
            ndarray: incrément de A pour chaque agent.
        """
        # Réduction du champ de signal vers la grille des agents (moyenne sur blocs)
        agent_signal = downscale_array(self.signal, self.agent_ratio)
        # Récupération de la valeur locale du signal pour chaque agent
        local_signal = agent_signal[self.coord[0, :], self.coord[1, :]]
        # Clip du signal local pour éviter des ratios trop extrêmes
        max_signal = 1e6 * self.c_params['Kd']
        min_signal = -0.999 * self.c_params['Kd']
        local_signal = np.clip(local_signal, min_signal, max_signal)
        # Calcul du ratio signal/Kd
        ratio = local_signal / self.c_params['Kd']
        # Clamp de sécurité sur le ratio
        ratio = np.clip(ratio, -0.999, 1e6)
        # Calcul de la dérivée de A (dynamique interne + contribution du signal)
        dA = ((self.A - (self.A**3) / 3 - self.R) +
              (self.c_params['a'] * np.log1p(ratio))) * self.dt
        return np.nan_to_num(dA, nan=0.0)

    def getdR(self):
        """
        Calcule la dérivée de R pour chaque agent.
        
        Returns:
            ndarray: incrément de R pour chaque agent.
        """
        dR = ((self.A - self.c_params['gamma'] * self.R + self.c_params['c0'])
              * self.c_params['epsilon']) * self.dt
        return np.nan_to_num(dR, nan=0.0)

    def getLapC(self):
        """
        Calcule la laplacienne du champ de signal.
        
        Returns:
            ndarray: laplacien du signal sous forme d'une grille 2D.
        """
        flat_signal = self.signal.flatten()
        laplacian = (1 / (self.g_params['dx']**2)) * self.lap_mat.dot(flat_signal)
        return laplacian.reshape(self.signal.shape)

    def update(self):
        """
        Met à jour l'état des agents et le champ de signal pour un pas de temps.
        
        Cette méthode :
            - Met à jour les états A et R des agents.
            - Calcule l'incrément du signal via diffusion et réaction.
            - Applique les flux aux bords.
        
        Returns:
            ndarray: nouveau champ de signal.
        """
        self.ts += self.dt  # Incrémentation du temps courant
        self.dA = self.getdA()  # Calcul de la dérivée de A
        if self.noise_flag:
            # Ajout de bruit à la dynamique de A
            self.dA += self.c_params['sigma'] * np.random.normal(
                loc=0.0, scale=np.sqrt(self.dt), size=self.A.shape)
        self.dR = self.getdR()  # Calcul de la dérivée de R

        # Mise à jour de la grille des agents
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        # Calcul de l'incrément du signal via diffusion-réaction
        self.dsignal = addcD(
            self.dt,
            self.getLapC(),
            self.g_params['D_sig'],
            self.c_params['aPDE'],
            self.c_params['rho'],
            self.c_params['a0'],
            self.c_params['D'],
            self.signal,
            np.heaviside(self.A_grid_big, 0.5)
        )

        # Mise à jour des états et du signal
        self.A += self.dA
        self.R += self.dR
        self.signal += self.dsignal

        # Application des flux aux bords
        dx = self.g_params['dx']
        self.signal[0, 1:-1]   -= self.fluxes[0] / dx
        self.signal[-1, 1:-1]  -= self.fluxes[1] / dx
        self.signal[1:-1, 0]   -= self.fluxes[2] / dx
        self.signal[1:-1, -1]  -= self.fluxes[3] / dx
        self.signal[0, 0]      -= (self.fluxes[0] + self.fluxes[2]) / (2 * dx)
        self.signal[-1, 0]     -= (self.fluxes[1] + self.fluxes[2]) / (2 * dx)
        self.signal[0, -1]     -= (self.fluxes[0] + self.fluxes[3]) / (2 * dx)
        self.signal[-1, -1]    -= (self.fluxes[1] + self.fluxes[3]) / (2 * dx)
        return self.signal

# =============================================================================
# FONCTION PRINCIPALE DE SIMULATION ET TRAÇAGE
# =============================================================================

def run_simulation():
    """
    Exécute la simulation de propagation d'AMPc et sauvegarde périodiquement les images 
    du champ de signal. En outre, enregistre et trace les valeurs de A et R pour la première 
    cellule en fonction du temps.
    """
    # Paramètres de la simulation
    T = 10              # Temps total de simulation (unités arbitraires)
    save_every = 1      # Intervalle de sauvegarde (en temps)
    sim = DictyPop(T, save_every)  # Initialisation de la simulation avec les paramètres par défaut
    steps = sim.Tsteps

    # Création du répertoire de sortie pour les images
    output_dir = "output_files"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Listes pour enregistrer le temps, A et R pour la cellule 0
    time_record = []
    A_record = []
    R_record = []

    print(f"Début de la simulation sur {steps} étapes (dt = {sim.dt:.4f}) ...")
    for step in range(steps):
        S = sim.update()  # Mise à jour de la simulation

        # Enregistrement du temps courant et des valeurs de A et R pour la cellule d'indice 0
        time_record.append(sim.ts)
        A_record.append(sim.A[0])
        R_record.append(sim.R[0])

        # Sauvegarde d'une image du champ de signal tous les 100 pas
        if step % 100 == 0:
            plt.figure()
            plt.imshow(S, cmap='viridis', origin='lower')
            plt.title(f"Signal à t = {sim.ts:.2f}")
            plt.colorbar(label="Signal")
            filename = os.path.join(output_dir, f"signal_{step:05d}.png")
            plt.savefig(filename)
            plt.close()
            print(f"Étape {step}/{steps} – image sauvegardée : {filename}")

    print("Simulation terminée.")

    # Tracé de la dynamique de A et R en fonction du temps pour la cellule 0
    plt.figure(figsize=(8, 4))
    plt.plot(time_record, A_record, label="A (cellule 0)", color='blue')
    plt.plot(time_record, R_record, label="R (cellule 0)", color='red')
    plt.xlabel("Temps")
    plt.ylabel("Valeur d'état")
    plt.title("Dynamique de A et R pour la cellule 0")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "cellule_0_dynamique.png"))
    plt.close()
    print(f"Tracé de A et R pour la cellule 0 sauvegardé dans {output_dir}/cellule_0_dynamique.png")

# =============================================================================
# LANCEMENT DE LA SIMULATION
# =============================================================================

if __name__ == "__main__":
    run_simulation()

Début de la simulation sur 1600 étapes (dt = 0.0063) ...
Étape 0/1600 – image sauvegardée : output_files/signal_00000.png


/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_20614/592122618.py:476: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


Étape 100/1600 – image sauvegardée : output_files/signal_00100.png
Étape 200/1600 – image sauvegardée : output_files/signal_00200.png
Étape 300/1600 – image sauvegardée : output_files/signal_00300.png
Étape 400/1600 – image sauvegardée : output_files/signal_00400.png
Étape 500/1600 – image sauvegardée : output_files/signal_00500.png
Étape 600/1600 – image sauvegardée : output_files/signal_00600.png
Étape 700/1600 – image sauvegardée : output_files/signal_00700.png
Étape 800/1600 – image sauvegardée : output_files/signal_00800.png
Étape 900/1600 – image sauvegardée : output_files/signal_00900.png
Étape 1000/1600 – image sauvegardée : output_files/signal_01000.png
Étape 1100/1600 – image sauvegardée : output_files/signal_01100.png
Étape 1200/1600 – image sauvegardée : output_files/signal_01200.png
Étape 1300/1600 – image sauvegardée : output_files/signal_01300.png
Étape 1400/1600 – image sauvegardée : output_files/signal_01400.png
Étape 1500/1600 – image sauvegardée : output_files/signal

/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_20614/592122618.py:488: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 4))


In [47]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# @author: Alexander Golden

### libraries import ###
from __future__ import division  # Doit être le premier import
import numpy as np
import matplotlib
import random as rand
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import math as math
import os
import sys
import scipy.stats as sc
import csv
import pandas as pd
import torch
import time
import scipy as sp
import scipy.sparse as sparse
from numba import njit, float64
from matplotlib.colors import LogNorm
import itertools as it
import pickle as pkl
import argparse

# Paramètres par défaut
dx = 0.5
ll = 50
agent_dim = 2 * dx

cell_params_default = {
    'c0': 1,
    'a': 0.05,
    'gamma': 0.5,
    'Kd': 10**(-5),
    'sigma': 0.15,
    'epsilon': 0.2,
    'cStim': 100,
    'aPDE': 10,
    'rho': 1,
    'D': 1000,
    'a0': 1,
    'af': 0
}

grid_params_default = {
    'dx': dx,
    'D_sig': 5,
    'box_size_x': ll,
    'box_size_y': ll,
    'agent_dim': agent_dim,
    'num_agents': 15000
}

### 'signaling functions' ###
@njit(float64[:,:](float64,float64[:,:],float64,float64,float64,float64,float64,float64[:,:],float64[:,:]))
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    return dt*(Dsig*lap - J*signal + rho*a0) + dt*(rho*D*A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    # Get linear indices to be used as IDs with bincount
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0]*shape[1]).reshape(shape)

def scale(A, B, k):
    # fill A with B scaled by k
    Y = A.shape[0]
    X = A.shape[1]
    for y in range(0, k):
        for x in range(0, k):
            A[y:Y:k, x:X:k] = B
    return A

def scale1D(A, B, k):
    # fill A with B scaled by k
    Y = A.shape[0]
    for y in range(0, k):
        A[y:Y:k] = B
    return A

def scale_down_mat_sp(current_shape, scale):
    # Generates a matrix that takes a matrix of one size and scales it down by
    # a factor given by integer argument "scale" by first flattening the
    # original matrix
    assert current_shape[0]//scale == current_shape[0]/scale, "Shape not divisible by scale"
    assert current_shape[1]//scale == current_shape[1]/scale, "Shape not divisible by scale"
    
    out_shape = (current_shape[0]//scale, current_shape[1]//scale)
    line = np.array(([1/scale**2]*scale+[0]*(current_shape[0]-scale))*scale+[0]*(
    current_shape[0]*(current_shape[1]-scale)))
    
    out_mat = sp.sparse.bsr_matrix(line)
    for i in range(out_shape[0]-1):
        newline = np.roll(line, scale*(i+1))
        out_mat = sp.sparse.vstack([out_mat, newline])
    
    block = out_mat.todense()
    for i in range(out_shape[1]-1):
        newblock = sp.sparse.bsr_matrix(np.roll(block, scale*current_shape[0]*(i+1), 1))
        out_mat = sp.sparse.vstack([out_mat, newblock])
    return sparse.csr_matrix(out_mat)

def calc_square_laplacian_noflux_matrix(size):
    #set up basic laplacian block diagonal matrix
    xDiff = np.zeros((size[1]+1, size[1]))
    ix, jx = np.indices(xDiff.shape)
    xDiff[ix==jx] = 1
    xDiff[ix==jx+1] = -1
    yDiff = np.zeros((size[0]+1, size[0]))
    iy, jy = np.indices(yDiff.shape)
    yDiff[iy==jy] = 1
    yDiff[iy==jy+1] = -1
    Ax = sparse.dia_matrix(-np.matmul(np.transpose(xDiff), xDiff))
    Ay = sparse.dia_matrix(-np.matmul(np.transpose(yDiff), yDiff))
    lap = sparse.kron(Ay, sparse.eye(size[1]), format='csr') + sparse.kron(sparse.eye(size[0]), Ax, format='csr')
    #set up boundary conditions
    lap += sparse.diags([2]+[1]*(size[1]-2)+[2]+([1]+[0]*(size[1]-2)+[1])*(size[0]-2)+[2]+[1]*(size[1]-2)+[2])
    lap = sparse.bsr_matrix(lap)
    return lap

def lapMatGivenFlux(size, fluxes):
    return False

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

### Population of signaling cells ###
class dictyPop:
    def __init__(self, T, save_every, g_params=grid_params_default, c_params=cell_params_default, 
                 noise=True, progress_report=True, save_data=True):
        self.g_params = g_params
        self.c_params = c_params
        self.T = T
        self.ts = 0
        self.dt = self.g_params['dx']**2/(8*self.g_params['D_sig'])
        self.Tsteps = int(self.T/self.dt)
        self.signal_size = (int(self.g_params['box_size_x']/self.g_params['dx']),
                            int(self.g_params['box_size_y']/self.g_params['dx']))
        self.fluxes = np.zeros(4)
        self.p_r = progress_report
        self.save_every = save_every
        self.save_data = save_data
        self.agent_ratio = int(self.g_params['agent_dim']/self.g_params['dx'])
        self.agent_size = (max(1, int(self.g_params['box_size_x']/self.g_params['agent_dim'])),
                           max(1, int(self.g_params['box_size_y']/self.g_params['agent_dim'])))
        self.signal = np.zeros(self.signal_size)
        self.noise_flag = noise
        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, g_params['num_agents']))
        self.A = cell_state[0,:]
        self.R = cell_state[1,:]
        self.D = float(self.c_params['D'])
        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)
        self.interp_mat = scale_down_mat_sp(self.signal_size, self.agent_ratio)
        self.A_grid = np.zeros((self.signal_size[0]//self.agent_ratio, 
                               self.signal_size[1]//self.agent_ratio))
        self.A_grid_big = np.zeros(self.signal.shape)
        self.coord = np.zeros((2, self.g_params['num_agents']), dtype='int')
        
        for agent in range(self.g_params['num_agents']):
            self.coord[:, agent] = np.array([
                int(rand.random()*self.g_params['box_size_x']/max(self.g_params['agent_dim'], self.g_params['dx'])),
                int(rand.random()*self.g_params['box_size_y']/max(self.g_params['agent_dim'], self.g_params['dx']))
            ])
        
        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            self.lap_mat = sparse.diags([np.ones(max(self.signal_size)-1), -2*np.ones(max(self.signal_size)), 
                                        np.ones(max(self.signal_size)-1)], [-1, 0, 1], format='csr')
            self.lap_mat += sparse.diags(np.array([1]+[0]*(max(self.signal_size)-2)+[1]), format='csr')
        
        if self.save_data is True:
            self.A_saved = np.zeros((self.signal_size[0]//self.agent_ratio,
                                    self.signal_size[1]//self.agent_ratio,
                                    (self.T//save_every)+1))
            self.R_saved = np.zeros((self.signal_size[0]//self.agent_ratio,
                                    self.signal_size[1]//self.agent_ratio,
                                    (self.T//save_every)+1))
            self.S_saved = np.zeros((self.signal_size[0],
                                    self.signal_size[1],
                                    (self.T//save_every)+1))
            self.A_saved[:,:,0] = self.getAGrid()
            self.R_saved[:,:,0] = self.getRGrid()
            self.S_saved[:,:,0] = self.signal
    
    def getAGrid(self):
        return accumulate_arr(self.coord, self.A, self.A_grid.shape)
    
    def getRGrid(self):
        return accumulate_arr(self.coord, self.R, self.A_grid.shape)
    
    def setSignal(self, signal):
        assert (self.signal.shape is signal.shape) and (self.signal.dtype is signal.dtype), "Input signal incorrect shape or type"
        self.signal = np.array(signal)
    
    def setCellState(self, state):
        assert (self.A.shape == state[:,0].shape) and (self.A.dtype == state.dtype), "Input cell state incorrect shape or type"
        self.A = np.array(state[:,0])
        self.R = np.array(state[:,1])
    
    def setFluxes(self, fluxes):
        assert (self.fluxes.shape is fluxes.shape) and (self.fluxes.dtype is fluxes.dtype), "Input fluxes incorrect shape or type"
        self.fluxes = np.array(fluxes)
    
    def getdA(self):
        return ((self.A-(self.A*self.A*self.A)/3 - self.R))*self.dt \
               + ((self.c_params['a']*np.log1p(np.reshape(
                  self.interp_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
                  self.agent_size)[self.coord[0,:], self.coord[1,:]]/self.c_params['Kd'])))*self.dt
    
    def getdR(self):
        return (((self.A-self.c_params['gamma']*self.R)+self.c_params['c0'])
                *self.c_params['epsilon'])*self.dt
    
    def getdC(self):
        #get laplacian for 0 flux
        laplacian = 1/(self.g_params['dx']**2)*np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1]))
        
        #set fluxes
        self.signal[0,1:-1] -= self.fluxes[0]/self.g_params['dx']
        self.signal[-1,1:-1] -= self.fluxes[1]/self.g_params['dx']
        self.signal[1:-1,0] -= self.fluxes[2]/self.g_params['dx']
        self.signal[1:-1,-1] -= self.fluxes[3]/self.g_params['dx']
        self.signal[0,0] -= (self.fluxes[0]+self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[-1,0] -= (self.fluxes[1]+self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[0,-1] -= (self.fluxes[0]+self.fluxes[3])/(2*self.g_params['dx'])
        self.signal[-1,-1] -= (self.fluxes[1]+self.fluxes[3])/(2*self.g_params['dx'])
        
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        
        return self.dt*(self.g_params['D_sig']*laplacian - self.c_params['aPDE']*self.signal 
                        + self.c_params['rho']*self.c_params['a0']) \
               + self.dt*(self.c_params['rho']*self.c_params['D']*np.heaviside(self.A_grid_big, 0.5))
    
    def getLapC(self):
        return 1/(self.g_params['dx']**2)*np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1]))
    
    def update(self):
        self.ts += self.dt
        self.dA = self.getdA()
        if self.noise_flag:
            self.dA += self.c_params['sigma']*np.random.normal(
                loc=0.0, scale=np.sqrt(self.dt), size=self.A.shape)
        self.dR = self.getdR()
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        
        self.dsignal = addcD(self.dt, self.getLapC(), self.g_params['D_sig'], 
                           self.c_params['aPDE'], self.c_params['rho'], 
                           self.c_params['a0'], self.c_params['D'],
                           self.signal, np.heaviside(self.A_grid_big, 0.5))
        
        self.A += self.dA
        self.R += self.dR
        self.signal += self.dsignal
        
        self.signal[0,1:-1] -= self.fluxes[0]/self.g_params['dx']
        self.signal[-1,1:-1] -= self.fluxes[1]/self.g_params['dx']
        self.signal[1:-1,0] -= self.fluxes[2]/self.g_params['dx']
        self.signal[1:-1,-1] -= self.fluxes[3]/self.g_params['dx']
        self.signal[0,0] -= (self.fluxes[0]+self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[-1,0] -= (self.fluxes[1]+self.fluxes[2])/(2*self.g_params['dx'])
        self.signal[0,-1] -= (self.fluxes[0]+self.fluxes[3])/(2*self.g_params['dx'])
        self.signal[-1,-1] -= (self.fluxes[1]+self.fluxes[3])/(2*self.g_params['dx'])
        
        # Mettre à jour les données sauvegardées si nécessaire
        if self.save_data and self.ts % self.save_every == 0:
            idx = int(self.ts // self.save_every)
            if idx < self.A_saved.shape[2]:
                self.A_saved[:,:,idx] = self.getAGrid()
                self.R_saved[:,:,idx] = self.getRGrid()
                self.S_saved[:,:,idx] = self.signal
        
        return self.signal

### Run the script and produce timeseries of signal intensity ###
def timeseries_a(a, t0, tf, freq, save=100, size=10):
    # Cell and grid parameter
    path = 'output_files/'
    N = int(15000)  # Population size
    cell_params = cell_params_default.copy()
    cell_params['a'] = a
    
    noise = True
    pop = dictyPop(tf, freq, c_params=cell_params, g_params=grid_params_default, noise=noise)
    pop.A = np.zeros(pop.A.shape)
    pop.R = np.zeros(pop.R.shape)
    S = pop.update()
    
    num_regions = int((len(S)/size)**2)
    TimeSeries = np.full((num_regions, 1), np.nan)
    
    for i in range(t0, tf):
        S = pop.update()
        
        if i % 100 == 0:
            plt.matshow(S)
            plt.title("t="+str(i))
            print(path)
            plt.savefig(path + str(i))
            plt.close()
        
        if i % save == 0:
            NewCol = []
            for i in range(0, len(S), size):
                for j in range(0, len(S), size):
                    if i+size <= len(S) and j+size <= len(S):
                        mean_val = np.mean(S[i:i+size, j:j+size])
                        if np.mean(S) > 0:
                            NewCol.append(mean_val / np.mean(S))
                        else:
                            NewCol.append(mean_val)
            
            if len(NewCol) == num_regions:
                NCol = pd.DataFrame({'col': NewCol})
                TimeSeries = np.column_stack((TimeSeries, NCol))
    
    return TimeSeries

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Script pour simuler la dynamique d'une population de cellules avec signalisation chimique.
Auteur: Alexander Golden

Ce script simule l'évolution d'un champ de signal chimique (par exemple, AMPc) diffusif couplé à la dynamique cellulaire.
Les résultats (images du champ de signal et séries temporelles) sont sauvegardés dans le dossier 'output_files'.
"""

### Import des bibliothèques ###
from __future__ import division  # Pour garantir la division réelle (utile en Python 2)
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Utilisation d'un backend non interactif pour sauvegarder les figures
import matplotlib.pyplot as plt
import math
import os
import sys
import scipy.stats as sc
import csv
import pandas as pd
import torch
import time
import scipy as sp
import scipy.sparse as sparse
from numba import njit, float64
from matplotlib.colors import LogNorm
import itertools as it
import random as rand
import pickle as pkl
import argparse

### Paramètres par défaut ###
dx = 0.5              # Pas spatial (distance entre deux points de la grille)
ll = 50               # Taille de la boîte (domaine de simulation en x et y)
agent_dim = 2 * dx    # Dimension caractéristique d'un agent

# Paramètres cellulaires par défaut
cell_params_default = {
    'c0': 1,            # Terme constant dans la dynamique cellulaire
    'a': 0.05,          # Coefficient d'activation (sensibilité au signal)
    'gamma': 0.5,       # Coefficient d'inhibition (régulation négative)
    'Kd': 10**(-5),     # Constante de dissociation (sensibilité)
    'sigma': 0.15,      # Amplitude du bruit dans la dynamique
    'epsilon': 0.2,     # Échelle temporelle pour la variable R
    'cStim': 100,       # Niveau de stimulation externe
    'aPDE': 10,         # Coefficient de dégradation du signal
    'rho': 1,           # Taux de production du signal
    'D': 1000,          # Coefficient additionnel lié à la production par les agents
    'a0': 1,            # Terme constant de production du signal
    'af': 0             # Paramètre additionnel (souvent inutilisé)
}

# Paramètres de la grille par défaut
grid_params_default = {
    'dx': dx,              # Pas spatial
    'D_sig': 5,            # Coefficient de diffusion du signal
    'box_size_x': ll,      # Taille du domaine en x
    'box_size_y': ll,      # Taille du domaine en y
    'agent_dim': agent_dim,  # Dimension caractéristique d'un agent
    'num_agents': 15000    # Nombre total d'agents (cellules)
}

### Fonctions de signalisation et utilitaires ###

@njit(float64[:,:](float64, float64[:,:], float64, float64, float64, float64, float64, float64[:,:], float64[:,:]))
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    """
    Calcule l'incrément du signal chimique (dC) sur un pas de temps dt en combinant diffusion et réaction.
    
    dC = dt*(Dsig*lap - J*signal + rho*a0) + dt*(rho*D*A_grid_hsided)
    
    Parameters:
      dt (float): Pas de temps.
      lap (ndarray): Laplacien du signal.
      Dsig (float): Coefficient de diffusion.
      J (float): Coefficient de réaction (aPDE).
      rho (float): Taux de production.
      a0 (float): Terme constant de production.
      D (float): Coefficient additionnel pour la production par agents.
      signal (ndarray): Champ de signal actuel.
      A_grid_hsided (ndarray): Grille d'agents après application d'une fonction seuil.
    
    Returns:
      ndarray: Incrément du signal.
    """
    return dt * (Dsig * lap - J * signal + rho * a0) + dt * (rho * D * A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    """
    Accumule les valeurs d'un tableau d'agents dans une grille 2D en fonction de leurs coordonnées.
    
    Parameters:
      coord (ndarray): Coordonnées des agents, de forme (2, nombre_d'agents).
      arr (ndarray): Valeurs associées à chaque agent.
      shape (tuple): Dimensions (lignes, colonnes) de la grille.
    
    Returns:
      ndarray: Grille 2D avec les valeurs accumulées.
    """
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0]*shape[1]).reshape(shape[0], shape[1])

def scale(A, B, k):
    """
    Remplit la matrice A avec la matrice B répétée en blocs de taille k x k.
    
    Parameters:
      A (ndarray): Matrice de destination.
      B (ndarray): Matrice source.
      k (int): Facteur d'échelle (taille du bloc).
    
    Returns:
      ndarray: Matrice A modifiée.
    """
    Y, X = A.shape
    for y in range(0, k):
        for x in range(0, k):
            A[y:Y:k, x:X:k] = B
    return A

def scale1D(A, B, k):
    """
    Version 1D de la fonction scale.
    
    Parameters:
      A (ndarray): Vecteur de destination.
      B (ndarray): Vecteur source.
      k (int): Facteur d'échelle.
    
    Returns:
      ndarray: Vecteur A modifié.
    """
    Y = A.shape[0]
    for y in range(0, k):
        A[y:Y:k] = B
    return A

def scale_down_mat_sp(current_shape, scale):
    """
    Génère une matrice creuse permettant de réduire une matrice d'origine par un facteur 'scale'
    en effectuant la moyenne sur des blocs.
    
    Parameters:
      current_shape (tuple): Dimensions de la matrice d'origine.
      scale (int): Facteur de réduction (doit diviser exactement les dimensions).
    
    Returns:
      csr_matrix: Matrice de downscaling.
    """
    assert current_shape[0] // scale == current_shape[0] / scale, "Shape not divisible by scale"
    assert current_shape[1] // scale == current_shape[1] / scale, "Shape not divisible by scale"
    out_shape = (current_shape[0] // scale, current_shape[1] // scale)
    line = np.array(([1/scale**2]*scale + [0]*(current_shape[0]-scale))*scale + [0]*(current_shape[0]*(current_shape[1]-scale)))
    out_mat = sp.sparse.bsr_matrix(line)
    for i in range(out_shape[0]-1):
        newline = np.roll(line, scale*(i+1))
        out_mat = sp.sparse.vstack([out_mat, newline])
    block = out_mat.todense()
    for i in range(out_shape[1]-1):
        newblock = sp.sparse.bsr_matrix(np.roll(block, scale*current_shape[0]*(i+1), axis=1))
        out_mat = sp.sparse.vstack([out_mat, newblock])
    return sparse.csr_matrix(out_mat)

def calc_square_laplacian_noflux_matrix(size):
    """
    Calcule la matrice laplacienne 2D avec conditions aux limites "no flux" pour une grille donnée.
    
    Parameters:
      size (tuple): Dimensions (lignes, colonnes) de la grille.
    
    Returns:
      csr_matrix: Matrice laplacienne compressée.
    """
    xDiff = np.zeros((size[1]+1, size[1]))
    ix, jx = np.indices(xDiff.shape)
    xDiff[ix == jx] = 1
    xDiff[ix == jx+1] = -1
    yDiff = np.zeros((size[0]+1, size[0]))
    iy, jy = np.indices(yDiff.shape)
    yDiff[iy == jy] = 1
    yDiff[iy == jy+1] = -1
    Ax = sparse.dia_matrix(-np.matmul(np.transpose(xDiff), xDiff))
    Ay = sparse.dia_matrix(-np.matmul(np.transpose(yDiff), yDiff))
    lap = sparse.kron(Ay, sparse.eye(size[1]), format='csr') + sparse.kron(sparse.eye(size[0]), Ax, format='csr')
    lap += sparse.diags([2] + [1]*(size[1]-2) + [2] + ([1] + [0]*(size[1]-2) + [1])*(size[0]-2) + [2] + [1]*(size[1]-2) + [2])
    lap = sparse.bsr_matrix(lap)
    return lap

def smooth(y, box_pts):
    """
    Lisse une série de données par convolution avec une fenêtre rectangulaire.
    
    Parameters:
      y (ndarray): Série de données.
      box_pts (int): Taille de la fenêtre.
    
    Returns:
      ndarray: Série lissée.
    """
    box = np.ones(box_pts) / box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

### Classe de simulation de la population de cellules ###
class dictyPop:
    def __init__(self, T, save_every, g_params=grid_params_default, c_params=cell_params_default, noise=True, progress_report=True, save_data=True):
        """
        Initialise la simulation de la population de cellules avec signalisation chimique.
        
        Parameters:
          T (float): Temps total de simulation.
          save_every (float): Intervalle de sauvegarde des données.
          g_params (dict): Paramètres de la grille.
          c_params (dict): Paramètres cellulaires.
          noise (bool): Active ou désactive le bruit dans la dynamique.
          progress_report (bool): Affiche la progression de la simulation.
          save_data (bool): Active la sauvegarde des données (pour séries temporelles).
        """
        self.g_params = g_params
        self.c_params = c_params
        self.T = T
        self.ts = 0  # Temps courant initialisé à 0
        self.dt = self.g_params['dx']**2 / (8 * self.g_params['D_sig'])  # Calcul du pas de temps basé sur la diffusion
        self.Tsteps = int(self.T / self.dt)
        # Dimensions de la grille du signal
        self.signal_size = (int(self.g_params['box_size_x'] / self.g_params['dx']),
                            int(self.g_params['box_size_y'] / self.g_params['dx']))
        self.fluxes = np.zeros(4)  # Initialisation des flux aux bords
        self.p_r = progress_report
        self.save_every = save_every
        self.save_data = save_data
        # Ratio pour downscaling (liaison entre la grille des agents et celle du signal)
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])
        self.agent_size = (max(1, int(self.g_params['box_size_x'] / self.g_params['agent_dim'])),
                           max(1, int(self.g_params['box_size_y'] / self.g_params['agent_dim'])))
        self.signal = np.zeros(self.signal_size)  # Champ de signal initialisé à zéro
        self.noise_flag = noise
        # Initialisation des états cellulaires (A et R) pour chaque agent, tirés d'une distribution normale
        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, g_params['num_agents']))
        self.A = cell_state[0, :]
        self.R = cell_state[1, :]
        self.D = float(self.c_params['D'])
        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)
        # Matrice d'interpolation pour downscaling du signal vers la grille des agents
        self.interp_mat = scale_down_mat_sp(self.signal_size, self.agent_ratio)
        # Grille pour accumuler les valeurs de A des agents
        self.A_grid = np.zeros((self.signal_size[0] // self.agent_ratio, self.signal_size[1] // self.agent_ratio))
        # Grille "agrandie" pour interpolation (utilisée dans la production du signal)
        self.A_grid_big = np.zeros(self.signal_size)
        # Coordonnées des agents dans la grille réduite
        self.coord = np.zeros((2, self.g_params['num_agents']), dtype='int')
        for agent in range(self.g_params['num_agents']):
            self.coord[:, agent] = np.array([
                int(rand.random() * ll / max(agent_dim, dx)),
                int(rand.random() * ll / max(agent_dim, dx))
            ])
        # Construction de la matrice laplacienne avec conditions "no flux"
        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            self.lap_mat = sparse.diags([np.ones(max(self.signal_size)-1),
                                         -2*np.ones(max(self.signal_size)),
                                         np.ones(max(self.signal_size)-1)],
                                        [-1, 0, 1], format='csr')
            self.lap_mat += sparse.diags(np.array([1] + [0]*(max(self.signal_size)-2) + [1]), format='csr')
        # Sauvegarde des données de la simulation (pour séries temporelles)
        if self.save_data is True:
            num_save = int(self.T // self.save_every) + 1  # Conversion en entier
            self.A_saved = np.zeros((self.signal_size[0] // self.agent_ratio,
                                     self.signal_size[1] // self.agent_ratio,
                                     num_save))
            self.R_saved = np.zeros((self.signal_size[0] // self.agent_ratio,
                                     self.signal_size[1] // self.agent_ratio,
                                     num_save))
            self.S_saved = np.zeros((self.signal_size[0],
                                     self.signal_size[1],
                                     num_save))
            self.A_saved[:, :, 0] = self.getAGrid()
            self.R_saved[:, :, 0] = self.getRGrid()
            self.S_saved[:, :, 0] = self.signal

    def getAGrid(self):
        """
        Retourne la grille 2D des valeurs de A accumulées selon les coordonnées des agents.
        
        Returns:
          ndarray: Grille des valeurs de A.
        """
        return accumulate_arr(self.coord, self.A, self.A_grid.shape)

    def getRGrid(self):
        """
        Retourne la grille 2D des valeurs de R accumulées selon les coordonnées des agents.
        
        Returns:
          ndarray: Grille des valeurs de R.
        """
        return accumulate_arr(self.coord, self.R, self.A_grid.shape)

    def setSignal(self, signal):
        """
        Définit le champ de signal.
        
        Parameters:
          signal (ndarray): Nouvelle matrice du signal.
        """
        assert (self.signal.shape is signal.shape) and (self.signal.dtype is signal.dtype), "Input signal incorrect shape or type"
        self.signal = np.array(signal)

    def setCellState(self, state):
        """
        Définit l'état cellulaire (A et R) pour tous les agents.
        
        Parameters:
          state (ndarray): Tableau de forme (2, nombre_d'agents) avec les valeurs de A et R.
        """
        assert (self.A.shape == state[:, 0].shape) and (self.A.dtype == state.dtype), "Input cell state incorrect shape or type"
        self.A = np.array(state[:, 0])
        self.R = np.array(state[:, 1])

    def setFluxes(self, fluxes):
        """
        Définit les flux appliqués aux bords.
        
        Parameters:
          fluxes (ndarray): Tableau de forme (4,) contenant les flux pour chaque bord.
        """
        assert (self.fluxes.shape is fluxes.shape) and (self.fluxes.dtype is fluxes.dtype), "Input fluxes incorrect shape or type"
        self.fluxes = np.array(fluxes)

    def getdA(self):
        """
        Calcule l'incrément de A pour chaque agent en combinant la dynamique interne (FitzHugh–Nagumo)
        et la contribution liée au signal local.
        
        Returns:
          ndarray: Incrément de A.
        """
        return ((self.A - (self.A*self.A*self.A)/3 - self.R)) * self.dt \
               + ((self.c_params['a'] * np.log1p(
                   np.reshape(self.interp_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
                              self.agent_size)[self.coord[0, :], self.coord[1, :]] / self.c_params['Kd']
               )))*self.dt

    def getdR(self):
        """
        Calcule l'incrément de R pour chaque agent selon la dynamique cellulaire.
        
        Returns:
          ndarray: Incrément de R.
        """
        return (((self.A - self.c_params['gamma']*self.R) + self.c_params['c0'])
                * self.c_params['epsilon'])*self.dt

    def getdC(self):
        """
        Calcule l'incrément de la concentration du signal en combinant diffusion et flux aux bords.
        
        Returns:
          ndarray: Incrément du signal.
        """
        laplacian = 1/(self.g_params['dx']**2) * np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1])
        )
        # Application manuelle des flux aux bords
        self.signal[0, 1:-1] -= self.fluxes[0] / self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1] / self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2] / self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3] / self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        return self.dt*(self.g_params['D_sig']*laplacian - self.c_params['aPDE']*self.signal + self.c_params['rho']*self.c_params['a0']) \
               + self.dt*(self.c_params['rho']*self.c_params['D']*np.heaviside(self.A_grid_big, 0.5))
    
    def getLapC(self):
        """
        Calcule le laplacien du champ de signal.
        
        Returns:
          ndarray: Laplacien sous forme d'une matrice 2D.
        """
        return 1/(self.g_params['dx']**2) * np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1])
        )
    
    def update(self):
        """
        Met à jour la simulation pour un pas de temps donné.
        
        Returns:
          ndarray: Nouveau champ de signal.
        """
        self.ts += self.dt  # Incrémente le temps courant
        self.dA = self.getdA()  # Calcul de l'incrément de A
        if self.noise_flag:
            # Ajout de bruit gaussien à A
            self.dA += self.c_params['sigma'] * np.random.normal(loc=0.0, scale=np.sqrt(self.dt), size=self.A.shape)
        self.dR = self.getdR()  # Calcul de l'incrément de R
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        self.dsignal = addcD(self.dt, self.getLapC(), self.g_params['D_sig'], 
                             self.c_params['aPDE'], self.c_params['rho'], 
                             self.c_params['a0'], self.c_params['D'],
                             self.signal, np.heaviside(self.A_grid_big, 0.5))
        # Mise à jour des états cellulaires et du signal
        self.A += self.dA
        self.R += self.dR
        self.signal += self.dsignal
        # Application des flux aux bords
        self.signal[0, 1:-1] -= self.fluxes[0] / self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1] / self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2] / self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3] / self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * self.g_params['dx'])
        return self.signal

### Fonction pour générer des séries temporelles ###
def timeseries_a(a, c0, t0, tf, freq, save=100, size=10):
    """
    Génère une série temporelle de l'intensité du signal en sauvegardant des images et en accumulant
    des moyennes locales.
    
    Parameters:
      a (float): Coefficient d'activation modifié.
      c0 (float): Terme constant modifié pour la dynamique cellulaire.
      t0 (int): Temps initial (pas de départ de la simulation).
      tf (int): Temps final (nombre total de pas).
      freq (float): Fréquence d'enregistrement (non utilisé ici).
      save (int): Intervalle de sauvegarde en nombre de pas.
      size (int): Taille du bloc pour le calcul des moyennes locales.
    
    Returns:
      ndarray: Série temporelle des intensités de signal normalisées.
    """
    path = 'output_files/'
    os.makedirs(path, exist_ok=True)  # Crée le dossier de sortie s'il n'existe pas
    N = int(15000)  # Taille de la population
    
    # Mise à jour des paramètres cellulaires
    cell_params = cell_params_default.copy()
    cell_params['c0'] = c0
    cell_params['a'] = a
    
    # Paramètres de la grille
    grid_params = {
        'dx': dx,
        'D_sig': 5,
        'box_size_x': ll,
        'box_size_y': ll,
        'agent_dim': agent_dim,
        'num_agents': N
    }
    
    noise = True
    # Initialisation de la population avec T=1 et save_every=1 pour cette fonction
    pop = dictyPop(1, 1, c_params=cell_params, g_params=grid_params, noise=noise)
    # Réinitialisation des états A et R à zéro
    pop.A = np.zeros(pop.A.shape)
    pop.R = np.zeros(pop.R.shape)
    S = pop.update()
    
    num_regions = int((len(S) / size)**2)
    TimeSeries = np.full((num_regions, 1), np.nan)
    
    # Boucle de simulation sur l'intervalle [t0, tf)
    for i in range(t0, tf):
        S = pop.update()
        NewCol = []
        # Sauvegarde d'une image tous les 100 pas
        if i % 100 == 0:
            plt.matshow(S)
            plt.title("t=" + str(i))
            plt.savefig(path + str(i) + '.png')
            plt.close()
        # Calcul des moyennes locales et mise à jour de la série temporelle tous les 'save' pas
        if i % save == 0:
            for i in range(0, len(S), size):
                for j in range(0, len(S), size):
                    if i + size <= len(S) and j + size <= len(S):
                        mean_val = np.mean(S[i:i+size, j:j+size])
                        if np.mean(S) > 0:
                            NewCol.append(mean_val / np.mean(S))
                        else:
                            NewCol.append(mean_val)
            if len(NewCol) == num_regions:
                NCol = pd.DataFrame({'col': NewCol})
                TimeSeries = np.column_stack((TimeSeries, NCol))
    
    return TimeSeries

### Exécution du script et production des séries temporelles ###
# Définir les variables manquantes
ll = 50  # Taille de la zone (domaine de simulation)
agent_dim = 2 * 0.5  # 0.5 est la valeur de dx
size = 50  # Taille pour calculer les moyennes locales

# Paramètres de la simulation
t0 = 0         # Pas de départ
tf = 100000    # Nombre total de pas
freq = 0.1     # Fréquence d'enregistrement (non utilisé ici)
a = 0.04       # Coefficient d'activation modifié
c0 = 0.5         # Terme constant modifié
# Lancer la simulation pour générer les séries temporelles
resultats = timeseries_a(a, c0, t0, tf, freq, 100, 10)

# Sauvegarder les résultats dans un fichier CSV
df = pd.DataFrame(resultats)
df.to_csv("timeseries_signal.csv", index=False)

# Visualiser les résultats avec un tracé
plt.plot(resultats)
plt.xlabel("Temps")
plt.ylabel("Intensité du signal")
plt.title("Séries temporelles de l'intensité du signal")
plt.show()

/opt/anaconda3/envs/particle_model/lib/python3.12/site-packages/scipy/sparse/_index.py:155: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/var/folders/sk/m9jkmr4n1k5dqc3wj_f53p4r0000gp/T/ipykernel_20614/3713436297.py:356: RuntimeWarning: invalid value encountered in log1p
  + ((self.c_params['a'] * np.log1p(


KeyboardInterrupt: 